In [2]:
import numpy as np
import Plot as plot
import math
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline
import time
import datetime
import pickle
import os
import scipy.spatial
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib as plt
import logging
from scipy import signal
import learningAlgs as classImportLA
import dataManipulation as dataMan
from itertools import permutations
import importlib
from datetime import timedelta
from multiprocessing import Pool
import multiprocessing
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.cluster import KMeans
import timeIntervalPlotter as intervalPlotter
import pysal
import warnings
import lumping_traditional as oldLumping
import boundaryFull_SS_WeightedLumping as WLumping
from importlib import reload
from scipy.stats import rayleigh
import dataManipulation as dataMan
import matplotlib.pyplot as plt
import processData as processData
import tensorflow as tf
# import tensorflow_probability as tfp
import random
from random import shuffle

warnings.filterwarnings('always')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# Fetching Data and Creating Transition Matrix

In [4]:
dataFrame = processData.processingData()
#address is: /home/netlab/Desktop/thesis/data/1node1-3-5/
#/home/netlab/Desktop/thesis/data/500f80271400/
data = dataFrame.copy() #copying the dataFrame to have a copy of not edited data

data = processData.dataFrameManipulation(data)

numberOfStates = 255
cuTrans_5min = processData.markovianTransitionMatrixDegree1(data, numberOfStates, "CU")
normalizedCuTrans_5min = processData.normalizingTransMatrix(cuTrans_5min)

the address of the collected data files (not alligned files or CSV files): /home/sepehr/thesis/data/500f80271400/
['500f80271400.txt']
here
500f80271400.txt is in csvChecker

we have the csv file: pulling out data

   col1                time  CU
0     0 2018-11-13 02:01:32  45
1     1 2018-11-13 02:01:38  45
2     2 2018-11-13 02:01:44  51
3     3 2018-11-13 02:01:50  53
4     4 2018-11-13 02:01:56  45
now we have the processed data from pandas
please enter how long would be the chunk minutes? 5
removing weekends from the data


# Merging Time Intervals For Training Days

In [90]:
# def dayCalculator(x):
#     years = x["time"].year - data["time"][0].year
#     months = x["time"].month - data["time"][0].month
#     days = x["time"].day - data["time"][0].day
#     return ((years*12*30) + (months*30) + days)
# data["day"] = data.apply(lambda x: dayCalculator(x), axis = 1)

# uniqueDays = []
# uniqueDays.extend(data.apply(lambda x: x["day"] if x["day"] not in uniqueDays else np.inf, axis = 1))
# uniqueDays = list(set(uniqueDays))
# random.Random(0).shuffle(uniqueDays)

# days30 = uniqueDays[0:30]
# data_within_30Days = data.loc[data["day"].isin(days30)]

# numberOfStates = 255
# cuTrans_5min = processData.markovianTransitionMatrixDegree1(data, numberOfStates, "CU")
# normalizedCuTrans_5min_30Days = processData.normalizingTransMatrix(cuTrans_5min_30Days)

# data_within_30Days_copy = data_within_30Days.copy()

In [247]:
steadyState_5min = np.zeros(shape=(normalizedCuTrans_5min.shape[0],
                                     normalizedCuTrans_5min.shape[1]))

for i in range(normalizedCuTrans_5min.shape[0]):
    steadyState_5min[i] = abs(pysal.spatial_dynamics.ergodic.steady_state(normalizedCuTrans_5min[i]))
    
correlationScore_numpy = np.zeros(shape = (normalizedCuTrans_5min.shape[0],normalizedCuTrans_5min.shape[0]))

listOfNeighbours = [0 for i in range(normalizedCuTrans_5min.shape[0])]

        

In [248]:
for i in range(normalizedCuTrans_5min.shape[0]):
    for j in range(normalizedCuTrans_5min.shape[0]):
        
        input1 = (steadyState_5min[i] - np.mean(steadyState_5min[i])) / (
            np.std(steadyState_5min[i]) * len(steadyState_5min[i]))

        input2 = (steadyState_5min[j] - np.mean(steadyState_5min[j])
                  ) / (np.std(steadyState_5min[j]))
        correlation = signal.correlate(input1, input2, mode="valid")
        correlationScore = correlation[int(np.floor(len(correlation)/2))]
        correlationScore_numpy[i][j] = correlationScore
        
for i in range(normalizedCuTrans_5min.shape[0]):
    correlationScore_numpy[i][i] = 0

In [249]:
takenIndices = []
checkIndices = []
print(correlationScore_numpy.shape[0])
i = 0
while i < (correlationScore_numpy.shape[0]/2):
    print(i)
    indices = np.unravel_index(np.argmax(correlationScore_numpy), correlationScore_numpy.shape)
    print(indices)
    if (int(indices[0]) in checkIndices) or (int(indices[1]) in checkIndices):
        correlationScore_numpy[int(indices[0])][int(indices[1])] = 0
        continue
    checkIndices.append(int(indices[0]))
    checkIndices.append(int(indices[1]))
    takenIndices.append(list(indices))
    correlationScore_numpy[indices[0]] = np.zeros(shape=[correlationScore_numpy.shape[0]])
    correlationScore_numpy[indices[1]] = np.zeros(shape=[correlationScore_numpy.shape[0]])
    i += 1

print(len(takenIndices))

steadyState_10min = np.zeros(shape=(len(takenIndices),normalizedCuTrans_5min.shape[1]))
for i in range(len(takenIndices)):
    print(i)
    data_correlation = data.loc[data["timeIndex"].isin(takenIndices[i])].copy()
    cuTran_corr = processData.markovianTransitionMatrixDegree1(data_correlation, numberOfStates, "CU")

    for x in range(cuTran_corr[-1].shape[0]):
        if np.sum(cuTran_corr[-1][x]) == 0:
            continue
        cuTran_corr[-1][x] = cuTran_corr[-1][x] / np.sum(cuTran_corr[-1][x])


    steadyState_10min[i] = abs(pysal.spatial_dynamics.ergodic.steady_state(cuTran_corr[-1]))

    correlationScore_numpy = np.zeros(shape =(steadyState_10min.shape[0],steadyState_10min.shape[0]))

for i in range(steadyState_10min.shape[0]):
    for j in range(steadyState_10min.shape[0]):
        input1 = (steadyState_10min[i] - np.mean(steadyState_10min[i])) / (
            np.std(steadyState_10min[i]) * len(steadyState_10min[i]))

        input2 = (steadyState_10min[j] - np.mean(steadyState_10min[j])
                  ) / (np.std(steadyState_10min[j]))
        correlation = signal.correlate(input1, input2, mode="valid")
        correlationScore = correlation[int(np.floor(len(correlation)/2))]
        correlationScore_numpy[i][j] = correlationScore

for i in range(steadyState_10min.shape[0]):
    correlationScore_numpy[i][i] = 0
        
# np.argmax(correlationScore_numpy)

288
0
(272, 270)
1
(26, 49)
2
(8, 27)
3
(1, 2)
4
(24, 55)
5
(18, 20)
6
(47, 46)
7
(14, 8)
7
(71, 72)
8
(41, 26)
8
(32, 36)
9
(262, 267)
10
(45, 26)
10
(7, 26)
10
(73, 75)
11
(33, 25)
12
(41, 40)
13
(48, 26)
13
(260, 261)
14
(29, 47)
14
(48, 49)
14
(31, 36)
14
(44, 49)
14
(29, 33)
14
(6, 41)
14
(85, 86)
15
(38, 39)
16
(42, 40)
16
(9, 73)
16
(43, 55)
16
(77, 80)
17
(6, 26)
17
(243, 247)
18
(0, 24)
18
(60, 71)
18
(7, 55)
18
(7, 41)
18
(42, 26)
18
(42, 43)
19
(58, 49)
19
(52, 8)
19
(31, 32)
19
(30, 31)
20
(7, 47)
20
(7, 49)
20
(5, 7)
21
(52, 26)
21
(4, 1)
21
(4, 23)
22
(44, 26)
22
(29, 46)
22
(52, 24)
22
(34, 25)
22
(9, 40)
22
(44, 7)
22
(78, 72)
22
(60, 64)
23
(10, 60)
23
(13, 63)
24
(14, 55)
24
(34, 33)
24
(54, 26)
24
(45, 43)
24
(259, 267)
24
(65, 45)
25
(258, 261)
25
(6, 49)
25
(21, 27)
25
(274, 260)
25
(29, 26)
25
(9, 75)
25
(3, 18)
25
(17, 32)
25
(29, 4)
25
(14, 24)
25
(44, 52)
26
(9, 68)
27
(35, 4)
27
(16, 19)
28
(54, 49)
28
(222, 221)
29
(259, 261)
29
(83, 79)
30
(246, 247)
30
(48,

(92, 46)
143
(92, 51)
143
(92, 4)
143
(92, 177)
143
(92, 49)
143
(92, 50)
143
(92, 41)
143
(92, 193)
143
(141, 202)
143
(92, 36)
143
(92, 34)
143
(92, 0)
143
(92, 27)
143
(92, 194)
143
(92, 35)
143
(92, 20)
143
(141, 201)
143
(92, 52)
143
(92, 33)
143
(92, 195)
143
(92, 179)
143
(92, 176)
143
(92, 53)
143
(92, 37)
143
(92, 29)
143
(92, 7)
143
(92, 185)
143
(92, 43)
143
(92, 127)
143
(92, 54)
143
(141, 110)
143
(92, 28)
143
(92, 188)
143
(92, 30)
143
(141, 206)
143
(92, 44)
143
(92, 173)
143
(92, 174)
143
(92, 25)
143
(92, 38)
143
(92, 186)
143
(92, 48)
143
(92, 31)
143
(92, 190)
143
(141, 207)
143
(92, 189)
143
(92, 171)
143
(92, 128)
143
(141, 105)
143
(92, 178)
143
(92, 182)
143
(92, 184)
143
(92, 192)
143
(92, 191)
143
(92, 183)
143
(92, 196)
143
(92, 180)
143
(141, 108)
143
(141, 104)
143
(141, 109)
143
(92, 187)
143
(92, 175)
143
(141, 209)
143
(92, 169)
143
(92, 170)
143
(92, 129)
143
(92, 181)
143
(92, 172)
143
(92, 152)
143
(92, 153)
143
(141, 208)
143
(141, 103)
143
(92, 131)


In [250]:
xyz = sorted([j for sub in takenIndices for j in sub])
for i in range(288):
    if i not in xyz:
        print(i)

In [270]:
import copy
# for i in range(steadyState_10min.shape[0]):
#     correlationScore_numpy[i][i] = 0
takenIndices_cpy = copy.deepcopy(takenIndices)
correlationScore_numpy_cpy= copy.deepcopy(correlationScore_numpy)
steadyState_10min_cpy = copy.deepcopy(steadyState_10min)



In [259]:
a = np.array([[0,1,2], [0,0,0]])
np.unravel_index(np.argmax(a), a.shape)

(0, 2)

In [271]:
np.unravel_index(np.argmax(correlationScore_numpy_cpy), correlationScore_numpy_cpy.shape)

(19, 8)

In [272]:
while np.max(correlationScore_numpy_cpy) > 0.98 and len(takenIndices_cpy) > 10:
    print(len(takenIndices_cpy), correlationScore_numpy_cpy.shape[0])
    indices = np.unravel_index(np.argmax(correlationScore_numpy_cpy), correlationScore_numpy_cpy.shape)

    print(indices, np.max(correlationScore_numpy_cpy))
    takenIndices_cpy[indices[0]].extend(takenIndices_cpy[indices[1]])
    takenIndices_cpy[indices[0]] = list(set(takenIndices_cpy[indices[0]]))
    print(takenIndices_cpy[indices[0]])
    
    data_correlation = data.loc[data["timeIndex"].isin(takenIndices_cpy[indices[0]])].copy()
    cuTran_corr = processData.markovianTransitionMatrixDegree1(data_correlation, numberOfStates, "CU")
    
    
    for x in range(cuTran_corr[-1].shape[0]):
        if np.sum(cuTran_corr[-1][x]) == 0:
            continue
        cuTran_corr[-1][x] = cuTran_corr[-1][x] / np.sum(cuTran_corr[-1][x])  
        
    steadyState_10min_cpy[indices[0]] = abs(pysal.spatial_dynamics.ergodic.steady_state(cuTran_corr[-1]))
    
    steadyState_10min_temp = np.zeros(shape=(len(takenIndices_cpy) - 1,normalizedCuTrans_5min.shape[1]))
    steadyState_10min_temp[:indices[1]] = steadyState_10min_cpy[:indices[1]]
    steadyState_10min_temp[indices[1]:] = steadyState_10min_cpy[indices[1] + 1 :]
    steadyState_10min_cpy = steadyState_10min_temp
    
    
    correlationScore_numpy_cpy = np.zeros(shape =(steadyState_10min_cpy.shape[0],steadyState_10min_cpy.shape[0]))

    for i in range(steadyState_10min_cpy.shape[0]):
        for j in range(steadyState_10min_cpy.shape[0]):
            input1 = (steadyState_10min_cpy[i] - np.mean(steadyState_10min_cpy[i])) / (
                np.std(steadyState_10min_cpy[i]) * len(steadyState_10min_cpy[i]))

            input2 = (steadyState_10min_cpy[j] - np.mean(steadyState_10min_cpy[j])
                      ) / (np.std(steadyState_10min_cpy[j]))
            correlation = signal.correlate(input1, input2, mode="valid")
            correlationScore = correlation[int(np.floor(len(correlation)/2))]
            correlationScore_numpy_cpy[i][j] = correlationScore
            
    for i in range(steadyState_10min_cpy.shape[0]):
        correlationScore_numpy_cpy[i][i] = 0
    
    del takenIndices_cpy[indices[1]]

#     steadyState_10min = np.zeros(shape=(len(takenIndices),normalizedCuTrans_5min.shape[1]))
#     for i in range(len(takenIndices)):
#         print(i)
#         data_correlation = data.loc[data["timeIndex"].isin(takenIndices[i])].copy()
#         cuTran_corr = processData.markovianTransitionMatrixDegree1(data_correlation, numberOfStates, "CU")

#         for x in range(cuTran_corr[-1].shape[0]):
#             if np.sum(cuTran_corr[-1][x]) == 0:
#                 continue
#             cuTran_corr[-1][x] = cuTran_corr[-1][x] / np.sum(cuTran_corr[-1][x])


#         steadyState_10min[i] = abs(pysal.spatial_dynamics.ergodic.steady_state(cuTran_corr[-1]))
                                   
#         correlationScore_numpy = np.zeros(shape =(steadyState_10min.shape[0],steadyState_10min.shape[0]))

#     for i in range(steadyState_10min.shape[0]):
#         for j in range(steadyState_10min.shape[0]):
#             input1 = (steadyState_10min[i] - np.mean(steadyState_10min[i])) / (
#                 np.std(steadyState_10min[i]) * len(steadyState_10min[i]))

#             input2 = (steadyState_10min[j] - np.mean(steadyState_10min[j])
#                       ) / (np.std(steadyState_10min[j]))
#             correlation = signal.correlate(input1, input2, mode="valid")
#             correlationScore = correlation[int(np.floor(len(correlation)/2))]
#             correlationScore_numpy[i][j] = correlationScore

144 144
(19, 8) 0.9968106021705235
[32, 36, 30, 31]
143 143
(4, 17) 0.99662734762802
[24, 42, 43, 55]
142 142
(4, 18) 0.9965721809679506
[5, 7, 42, 43, 55, 24]
141 141
(10, 4) 0.996555496876319
[33, 5, 7, 42, 43, 55, 24, 25]
140 140
(1, 45) 0.9963777580590734
[48, 49, 26, 58]
139 139
(6, 8) 0.9963319243969028
[72, 73, 75, 71]
138 138
(5, 9) 0.9961530791775165
[40, 41, 46, 47]
137 137
(7, 31) 0.9960390621016666
[259, 258, 267, 262]
136 136
(47, 2) 0.9959574352424471
[8, 27, 12, 21]
135 135
(41, 12) 0.9957911367888584
[247, 243, 238, 246]
134 134
(43, 11) 0.9956834014980986
[80, 76, 77, 78]
133 133
(0, 59) 0.9956755599137167
[272, 265, 274, 270]
132 132
(11, 44) 0.9956635642223296
[32, 36, 8, 12, 21, 27, 30, 31]
131 131
(2, 13) 0.9955313184900891
[64, 1, 2, 60]
130 130
(7, 10) 0.9954934999237985
[33, 5, 38, 7, 39, 42, 43, 55, 24, 25]
129 129
(9, 18) 0.9954831386742867
[83, 85, 86, 79]
128 128
(6, 8) 0.995444234573498
[258, 259, 260, 261, 262, 267]
127 127
(3, 10) 0.995326206590364
[18, 2

40 40
(7, 8) 0.9882088992431481
[208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 223, 224, 225, 226, 228, 229, 235, 239, 241]
39 39
(15, 23) 0.9880851828993538
[155, 132, 133, 135]
38 38
(19, 25) 0.988037486525222
[198, 199, 201, 202, 203, 204, 205, 206, 207, 190]
37 37
(21, 24) 0.988017748895135
[128, 136, 137, 138, 148, 149]
36 36
(11, 34) 0.9877395663149581
[97, 103, 104, 106, 107, 110, 111, 112, 113, 114, 115, 116, 117, 118]
35 35
(13, 21) 0.9876090000996155
[128, 134, 136, 137, 138, 148, 149, 154]
34 34
(8, 33) 0.9875048274034048
[144, 145, 92, 141]
33 33
(16, 18) 0.9875013504375443
[192, 193, 195, 197, 180, 183, 185, 186, 187, 188, 189, 191]
32 32
(8, 20) 0.9874235948971724
[162, 141, 144, 145, 92, 159]
31 31
(30, 23) 0.9873232123928052
[166, 165, 158, 167]
30 30
(16, 26) 0.9869962497763487
[192, 193, 194, 195, 197, 180, 183, 184, 185, 186, 187, 188, 189, 191]
29 29
(2, 7) 0.9865536420648361
[208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 2

In [280]:
takenIndices_cpy[8]

[198, 199, 201, 202, 203, 204, 205, 206, 207, 190]

In [281]:
data.head()

,col1,time,CU,CU/255,timeIndex,weekDay,cluster,day
0,0,2018-11-12 19:01:32,45,0.176471,228,0,2.0,0
1,1,2018-11-12 19:01:38,45,0.176471,228,0,2.0,0
2,2,2018-11-12 19:01:44,51,0.200000,228,0,2.0,0
3,3,2018-11-12 19:01:50,53,0.207843,228,0,2.0,0
4,4,2018-11-12 19:01:56,45,0.176471,228,0,2.0,0


In [274]:
xyz = sorted([j for sub in takenIndices_cpy for j in sub])
for i in range(288):
    if i not in xyz:
        print(i)

In [282]:
for i in range(len(takenIndices_cpy)):
    for j in range(len(takenIndices_cpy[i])):
        data["cluster"].loc[data["timeIndex"] == takenIndices_cpy[i][j]] = i

In [287]:
data.tail()

,col1,time,CU,CU/255,timeIndex,weekDay,cluster,day
1562731,1562731,2019-08-06 10:34:08,51,0.200000,126,1,6,264
1562732,1562732,2019-08-06 10:34:14,40,0.156863,126,1,6,264
1562733,1562733,2019-08-06 10:34:20,40,0.156863,126,1,6,264
1562734,1562734,2019-08-06 10:34:26,51,0.200000,126,1,6,264
1562735,1562735,2019-08-06 10:34:32,48,0.188235,126,1,6,264


In [286]:
data["cluster"] = data["cluster"].astype(int)

In [288]:
data["timeIndex"] = data["cluster"]

# Done

In [16]:
takenIndices = []
for x in range(correlationScore_numpy.shape[0]):
    if x not in takenIndices:
        takenIndices.append(x)
        
        if correlationScore_numpy[x][np.argmax(correlationScore_numpy[x])] > 0.98:
            takenIndices.append(np.argmax(correlationScore_numpy[x]))
    
    for y in range(correlationScore_numpy.shape[0]):

0

In [19]:
np.argmax(correlationScore_numpy[0])

24

In [132]:
for i in range(normalizedCuTrans_5min.shape[0]):
    for j in range(normalizedCuTrans_5min.shape[0]):
        
        input1 = (steadyState_5min[i] - np.mean(steadyState_5min[i])) / (
            np.std(steadyState_5min[i]) * len(steadyState_5min[i]))

        input2 = (steadyState_5min[j] - np.mean(steadyState_5min[j])
                  ) / (np.std(steadyState_5min[j]))
        correlation = signal.correlate(input1, input2, mode="valid")
        correlationScore = correlation[int(np.floor(len(correlation)/2))]
        correlationScore_numpy[i][j] = correlationScore
        
i = 0
listOfNeighbours = []
takenIndexes = []
while i < correlationScore_numpy.shape[0]:
    if i in takenIndexes:
        i += 1
        continue

    listOfNeighbours.append([])
    listOfNeighbours[-1].append(i)
    takenIndexes.append(i)
    for j in range(i + 1, correlationScore_numpy.shape[0]):
        if (correlationScore_numpy[i][j] > 0.97) and (j not in takenIndexes):
            listOfNeighbours[-1].append(j)
            takenIndexes.append(j)

print(listOfNeighbours)
len(listOfNeighbours)
for i in range(len(listOfNeighbours)):
    for j in range(len(listOfNeighbours[i])):
        data["timeIndex"].loc[data["timeIndex"] == listOfNeighbours[i][j]] = i    

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 83, 85, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287], [82, 84, 86, 87, 88, 89, 90, 248, 250], [91, 92, 93, 94, 95, 96, 98, 99], [97, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 113, 114, 115, 116], [112, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 173, 176, 177, 197, 198, 199, 205], [127, 128, 129, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 167, 168, 169, 170, 172, 174, 175, 178, 179, 180, 181, 182], [130, 131, 142, 143, 164, 1

In [40]:
listOfNeighbours = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 83, 85, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287], [82, 84, 86, 87, 88, 89, 90, 248, 250], [91, 92, 93, 94, 95, 96, 98, 99], [97, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 113, 114, 115, 116], [112, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 173, 176, 177, 197, 198, 199, 205], [127, 128, 129, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 167, 168, 169, 170, 172, 174, 175, 178, 179, 180, 181, 182], [130, 131, 142, 143, 164, 165, 166], [144, 145], [171, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195], [196, 200, 201, 202, 203, 204, 206], [207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 221, 222, 223, 225, 226, 228, 229, 231, 232, 233, 234, 235], [220, 224, 227, 230, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 249, 251, 252]]

In [46]:
print(listOfNeighbours[0])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 83, 85, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287]


In [50]:
x30 = []
for i in range(len(listOfNeighbours)):
    x30.append([])
    for j in range(len(listOfNeighbours[i])):
        if np.floor(listOfNeighbours[i][j]*5/30) not in x30[i]:
            x30[i].append(np.floor(listOfNeighbours[i][j]*5/30))
print(x30)        

[[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0], [13.0, 14.0, 15.0, 41.0], [15.0, 16.0], [16.0, 17.0, 18.0, 19.0], [18.0, 19.0, 20.0, 21.0, 28.0, 29.0, 32.0, 33.0, 34.0], [21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0], [21.0, 23.0, 27.0], [24.0], [28.0, 30.0, 31.0, 32.0], [32.0, 33.0, 34.0], [34.0, 35.0, 36.0, 37.0, 38.0, 39.0], [36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0]]


# Calculating days and shuffling with seed to create Correlations

In [161]:
def dayCalculator(x):
    years = x["time"].year - data["time"][0].year
    months = x["time"].month - data["time"][0].month
    days = x["time"].day - data["time"][0].day
    return ((years*12*30) + (months*30) + days)
data["day"] = data.apply(lambda x: dayCalculator(x), axis = 1)

In [289]:
uniqueDays = []
uniqueDays.extend(data.apply(lambda x: x["day"] if x["day"] not in uniqueDays else np.inf, axis = 1))
uniqueDays = list(set(uniqueDays))
random.Random(0).shuffle(uniqueDays)

days30 = uniqueDays[0:30]
data_within_30Days = data.loc[data["day"].isin(days30)]

In [290]:
numberOfStates = 255
cuTrans_30Days = processData.markovianTransitionMatrixDegree1(data_within_30Days, numberOfStates, "CU")
normalizedCuTrans_30Days = processData.normalizingTransMatrix(cuTrans_30Days)

In [291]:
from scipy import signal
pd.options.mode.chained_assignment = None
steadyState_30days = np.zeros(shape=(normalizedCuTrans_30Days.shape[0],
                                     normalizedCuTrans_30Days.shape[1]))
for i in range(normalizedCuTrans_30Days.shape[0]):
    steadyState_30days[i] = abs(pysal.spatial_dynamics.ergodic.steady_state(normalizedCuTrans_30Days[i]))

# Calculating Correlations

In [292]:
for x in range(max(data["timeIndex"]) + 1):
    data["corr" + str(x + 1)] = np.inf
    
print(uniqueDays)
for j in uniqueDays:
    print(j)
    for i in range(max(data["timeIndex"]) + 1):
        data_correlation = data.loc[(data["timeIndex"] == i) & (data["day"] == j)].copy()
        if len(data_correlation) == 0:
            continue
        cuTran_corr = processData.markovianTransitionMatrixDegree1(data_correlation, numberOfStates, "CU")
        for x in range(cuTran_corr[-1].shape[0]):
            if np.sum(cuTran_corr[-1][x]) == 0:
                continue
            cuTran_corr[-1][x] = cuTran_corr[-1][x] / np.sum(cuTran_corr[-1][x])
        steadyState_corr = abs(pysal.spatial_dynamics.ergodic.steady_state(cuTran_corr[-1]))

        steadyState_corr_normalize = (steadyState_corr - np.mean(steadyState_corr)) / (
            np.std(steadyState_corr) * len(steadyState_corr))
        
        correlationDistance = []
        
        for x in range(max(data["timeIndex"]) + 1):
            ss_normalize = (steadyState_30days[x] - np.mean(steadyState_30days[x])) / (
                np.std(steadyState_30days[x]))
            data["corr" + str(x + 1)].loc[(data["timeIndex"] == i) & (data["day"] == j)] = signal.correlate(
                steadyState_corr_normalize, ss_normalize, mode="valid")[0]

[31, 10, 15, 9, 185, 17, 1, 252, 257, 195, 22, 205, 191, 167, 238, 2, 184, 246, 224, 210, 25, 8, 32, 14, 201, 147, 30, 146, 240, 161, 254, 236, 177, 264, 245, 141, 153, 219, 178, 18, 243, 216, 4, 28, 21, 29, 148, 223, 198, 211, 218, 189, 215, 35, 260, 183, 259, 0, 217, 154, 199, 253, 3, 258, 197, 251, 36, 164, 196, 250, 247, 202, 168, 11, 239, 163, 24, 225, 212, 149, 237, 16, 244, 160, 23, 208, 37, 222, 182, 203, 162, 190, 204, 209, 150, 7, 192, 188]
31
10
15
9
185
17
1
252
257
195
22
205
191
167
238
2
184
246
224
210
25
8
32
14
201
147
30
146
240
161
254
236
177
264
245
141
153
219
178
18
243
216
4
28
21
29
148
223
198
211
218
189
215
35
260
183
259
0
217
154
199
253
3
258
197
251
36
164
196
250
247
202
168
11
239
163
24
225
212
149
237
16
244
160
23
208
37
222
182
203
162
190
204
209
150
7
192
188


In [293]:
data.head()

,col1,time,CU,CU/255,timeIndex,weekDay,cluster,day,corr1,corr2,corr3,corr4,corr5,corr6,corr7,corr8,corr9,corr10
0,0,2018-11-12 19:01:32,45,0.176471,9,0,9,0,0.776963,0.806844,0.728127,0.73227,0.669533,0.812511,0.569408,0.518945,0.551643,0.555947
1,1,2018-11-12 19:01:38,45,0.176471,9,0,9,0,0.776963,0.806844,0.728127,0.73227,0.669533,0.812511,0.569408,0.518945,0.551643,0.555947
2,2,2018-11-12 19:01:44,51,0.200000,9,0,9,0,0.776963,0.806844,0.728127,0.73227,0.669533,0.812511,0.569408,0.518945,0.551643,0.555947
3,3,2018-11-12 19:01:50,53,0.207843,9,0,9,0,0.776963,0.806844,0.728127,0.73227,0.669533,0.812511,0.569408,0.518945,0.551643,0.555947
4,4,2018-11-12 19:01:56,45,0.176471,9,0,9,0,0.776963,0.806844,0.728127,0.73227,0.669533,0.812511,0.569408,0.518945,0.551643,0.555947


# Creating Staionary Process Values

In [294]:
copy_data = data.copy()

In [295]:
data = copy_data.copy()

In [296]:
data["logDiff"] = np.inf
data["log"] = np.inf
data["normalLogDiff"] = np.inf

In [297]:
max(data["timeIndex"])

9

In [298]:
for i in range(max(data["timeIndex"]) + 1):
    dataTimeIndex = data.loc[data["timeIndex"] == i]
    dataTimeIndex["log"] = np.log(dataTimeIndex["CU"])
    dataTimeIndex = dataTimeIndex.loc[dataTimeIndex["log"] != -np.inf]
    dataTimeIndex = dataTimeIndex.loc[dataTimeIndex["log"] != np.inf]
    dataTimeIndex = dataTimeIndex.dropna()
    
    dataTimeIndex["logDiff"] = dataTimeIndex["log"].diff()
    dataTimeIndex = dataTimeIndex.loc[dataTimeIndex["logDiff"] != -np.inf]
    dataTimeIndex = dataTimeIndex.loc[dataTimeIndex["logDiff"] != -np.inf]
    dataTimeIndex = dataTimeIndex.dropna()
    
    dataTimeIndex["normalLogDiff"] = dataTimeIndex["logDiff"] / np.max(dataTimeIndex["logDiff"])
    data.loc[data["timeIndex"] == i] = dataTimeIndex
    
data = data.loc[data["log"] != np.inf]
data = data.loc[data["log"] != -np.inf]
data = data.dropna()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeW

# Writing Feature Engineered DataFrame into a Pickle

In [299]:
data["timeIndex"] = data["timeIndex"].astype(int)

In [300]:
data.to_pickle("5minMerged_featureEngineer_correct.pickle")

In [301]:
data = pd.read_pickle("5minMerged_featureEngineer_correct.pickle")

# Lumping 25 Cons

In [302]:
def preparingMatrixForLumping(transitionMatrix):
    reload(oldLumping)
    percentageMatrix_list = bandwidthPercentage(transitionMatrix)
    zero_cols_rows = []
    #*************removing zeros from columns and rows (matrix reduction) ****************
    for i in range(len(transitionMatrix)):
        if (np.sum(transitionMatrix[i]) == 0) and (np.sum(transitionMatrix[:,i]) == 0):
            zero_cols_rows.append(i)


    irreducible_matrix = transitionMatrix.copy()
    for i in range(len(zero_cols_rows) - 1, -1, -1):
        irreducible_matrix = np.delete(irreducible_matrix, zero_cols_rows[i], axis = 0)
        irreducible_matrix = np.delete(irreducible_matrix, zero_cols_rows[i], axis = 1)
        percentageMatrix_list[zero_cols_rows[i]][2] = True


    for i in range(len(percentageMatrix_list) - 1, 0, -1):
        if (percentageMatrix_list[i][2] == True) and (percentageMatrix_list[i - 1][2] == True):
            percentageMatrix_list[i - 1][0].extend(percentageMatrix_list[i][0])
            percentageMatrix_list[i - 1][1] = percentageMatrix_list[i][1]
            del percentageMatrix_list[i]

    for i in range(len(irreducible_matrix)):
        if np.sum(irreducible_matrix[i], dtype = np.float32) != 1.0:
            print(np.sum(irreducible_matrix[i], dtype = np.float32))

    return percentageMatrix_list, irreducible_matrix

def bandwidthPercentage(vectorMatrix):
    percentageIncreament = (100 / vectorMatrix.shape[0])
    percentageMatrix = []
    maxPercentage = 0
    for j in range(vectorMatrix.shape[0]):
        maxPercentage += percentageIncreament
        percentageMatrix.append([[j], maxPercentage, False])

    return percentageMatrix

In [303]:
numberOfStates = 255
cuTrans_5min = processData.markovianTransitionMatrixDegree1(data, numberOfStates, "CU")
normalizedCuTrans_5min = processData.normalizingTransMatrix(cuTrans_5min)

## 7 Clusters

In [312]:
reload(oldLumping)

<module 'lumping_traditional' from '/home/sepehr/thesis/APDataML/lumping_traditional.py'>

Process ForkPoolWorker-871:
KeyboardInterrupt
Process ForkPoolWorker-870:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/pyth

In [313]:
for i in range(len(normalizedCuTrans_5min)):
    percentageMatrix_list, irreducible_matrix = preparingMatrixForLumping(normalizedCuTrans_5min[i])
    result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, False)
    nameResult = "/home/sepehr/thesis/APDataML/pickles/7merging_corrected_lumping" + str(i) + ".pickle"
    with open(nameResult, 'wb') as handle:
        pickle.dump(result, handle)
    namePercent = "/home/sepehr/thesis/APDataML/pickles/7merging_corrected_lumping_percentage_" + str(i) + ".pickle"
    with open(namePercent, 'wb') as handle:
        pickle.dump(percentageMatrix_list, handle)

0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
we have these many sectors to check:  882636
0.99999994
0.99999994
0.9999999
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  882636
1.0000001
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
we have these many sectors to check:  603164
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  882636
0.99999994
0.99999994
0.99999994
0.9999999
0.99999994
we have these many sectors to check:  56512
0.99999994
0.9999999
0.99999994
1.0000001
0.99999994
1.0000001
0.9999999
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
we have these many sectors to check:  882636
0.99999994
0.9999999
0.99999994
1.0000001
1.0000001
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
1.0000

In [325]:
arrayOfLumpes = []
indexesOfCandidates = [[] for i in range(10)]
for timeIndex in range(10):
    print(timeIndex)
    name = "/home/sepehr/thesis/APDataML/pickles/7merging_corrected_lumping" + str(timeIndex) + ".pickle"
    with open(name, 'rb') as handle:
        b = pickle.load(handle)
    
    if len(b) > 0:
        minDegree = np.inf        
        for i in range(len(b)):
            if b[i][0] < minDegree:
                minDegree = b[i][0]

        minError = np.inf
        for i in range(len(b)):
            if b[i][0] == minDegree and b[i][1] < minError:
                minError = b[i][1]

        for i in range(len(b)):
            if b[i][0] == minDegree and b[i][1] == minError:
                indexesOfCandidates[timeIndex].append(b[i])

name = "/home/sepehr/thesis/APDataML/7merging_corrected_lumping_Boundaries.pickle"
with open(name, 'wb') as handle:
    pickle.dump(indexesOfCandidates, handle) 

0
1
2
3
4
5
6
7
8
9


In [326]:
percentageArray = []
for i in range(10):
    name = "/home/sepehr/thesis/APDataML/pickles/7merging_corrected_lumping_percentage_" + str(i) + ".pickle"
    with open(name, 'rb') as handle:
        percentageArray.append(pickle.load(handle))

In [306]:
reload(oldLumping)

<module 'lumping_traditional' from '/home/sepehr/thesis/APDataML/lumping_traditional.py'>

In [ ]:
for i in range(len(normalizedCuTrans_5min)):
    percentageMatrix_list, irreducible_matrix = preparingMatrixForLumping(normalizedCuTrans_5min[i])
    result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, False)
    nameResult = "/home/sepehr/thesis/APDataML/pickles/6merging_corrected_lumping" + str(i) + ".pickle"
    with open(nameResult, 'wb') as handle:
        pickle.dump(result, handle)
    namePercent = "/home/sepehr/thesis/APDataML/pickles/6merging_corrected_lumping_percentage_" + str(i) + ".pickle"
    with open(namePercent, 'wb') as handle:
        pickle.dump(percentageMatrix_list, handle)

0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
we have these many sectors to check:  366134


In [327]:
arrayOfLumpes = []
indexesOfCandidates = [[] for i in range(10)]
for timeIndex in range(10):
    print(timeIndex)
    name = "/home/sepehr/thesis/APDataML/pickles/6merging_corrected_lumping" + str(timeIndex) + ".pickle"
    with open(name, 'rb') as handle:
        b = pickle.load(handle)
    
    if len(b) > 0:
        minDegree = np.inf        
        for i in range(len(b)):
            if b[i][0] < minDegree:
                minDegree = b[i][0]

        minError = np.inf
        for i in range(len(b)):
            if b[i][0] == minDegree and b[i][1] < minError:
                minError = b[i][1]

        for i in range(len(b)):
            if b[i][0] == minDegree and b[i][1] == minError:
                indexesOfCandidates[timeIndex].append(b[i])

name = "/home/sepehr/thesis/APDataML/6merging_corrected_lumping_Boundaries.pickle"
with open(name, 'wb') as handle:
    pickle.dump(indexesOfCandidates, handle) 

0
1
2
3
4
5
6
7
8
9


In [328]:
percentageArray = []
for i in range(10):
    name = "/home/sepehr/thesis/APDataML/pickles/6merging_corrected_lumping_percentage_" + str(i) + ".pickle"
    with open(name, 'rb') as handle:
        percentageArray.append(pickle.load(handle))

In [349]:
reload(oldLumping)

<module 'lumping_traditional' from '/home/sepehr/thesis/APDataML/lumping_traditional.py'>

In [351]:
for i in range(len(normalizedCuTrans_5min)):
    percentageMatrix_list, irreducible_matrix = preparingMatrixForLumping(normalizedCuTrans_5min[i])
    result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, False)
    nameResult = "/home/sepehr/thesis/APDataML/pickles/5merging_corrected_lumping" + str(i) + ".pickle"
    with open(nameResult, 'wb') as handle:
        pickle.dump(result, handle)
    namePercent = "/home/sepehr/thesis/APDataML/pickles/5merging_corrected_lumping_percentage_" + str(i) + ".pickle"
    with open(namePercent, 'wb') as handle:
        pickle.dump(percentageMatrix_list, handle)

0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
we have these many sectors to check:  13048
0.99999994
0.99999994
0.9999999
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  13048
1.0000001
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
we have these many sectors to check:  11610
1.0000001
1.0000001
1.0000001
0.99999994
0.9999999
0.99999994
0.99999994
0.9999999
1.0000001
we have these many sectors to check:  11351
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  13048
0.99999994
0.99999994
0.99999994
0.9999999
0.99999994
we have these many sectors to check:  5203
0.99999994
0.9999999
0.99999994
1.0000001
0.99999994
1.0000001
0.9999999
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
we have these many sectors to check:  13048
0.99999994
0.9999999
0.9

In [352]:
arrayOfLumpes = []
indexesOfCandidates = [[] for i in range(10)]
for timeIndex in range(10):
    print(timeIndex)
    name = "/home/sepehr/thesis/APDataML/pickles/5merging_corrected_lumping" + str(timeIndex) + ".pickle"
    with open(name, 'rb') as handle:
        b = pickle.load(handle)
    
    if len(b) > 0:
        minDegree = np.inf        
        for i in range(len(b)):
            if b[i][0] < minDegree:
                minDegree = b[i][0]

        minError = np.inf
        for i in range(len(b)):
            if b[i][0] == minDegree and b[i][1] < minError:
                minError = b[i][1]

        for i in range(len(b)):
            if b[i][0] == minDegree and b[i][1] == minError:
                indexesOfCandidates[timeIndex].append(b[i])

name = "/home/sepehr/thesis/APDataML/5merging_corrected_lumping_Boundaries.pickle"
with open(name, 'wb') as handle:
    pickle.dump(indexesOfCandidates, handle) 

0
1
2
3
4
5
6
7
8
9


In [353]:
percentageArray = []
for i in range(10):
    name = "/home/sepehr/thesis/APDataML/pickles/5merging_corrected_lumping_percentage_" + str(i) + ".pickle"
    with open(name, 'rb') as handle:
        percentageArray.append(pickle.load(handle))

# 6 Clusters

In [186]:
reload(oldLumping)

<module 'lumping_traditional' from '/home/sepehr/thesis/APDataML/lumping_traditional.py'>

In [181]:
for i in range(len(normalizedCuTrans_5min)):
    percentageMatrix_list, irreducible_matrix = preparingMatrixForLumping(normalizedCuTrans_5min[i])
    result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, False)
    nameResult = "/home/sepehr/thesis/APDataML/pickles/4normal_correlated_lumping_result_" + str(i) + ".pickle"
    with open(nameResult, 'wb') as handle:
        pickle.dump(result, handle)
    namePercent = "/home/sepehr/thesis/APDataML/pickles/4normal_correlated_lumping_percentage_" + str(i) + ".pickle"
    with open(namePercent, 'wb') as handle:
        pickle.dump(percentageMatrix_list, handle)

1.0000001
0.99999994
0.99999994
0.9999999
1.0000001
0.99999994
0.99999994
0.99999994
1.0000001
we have these many sectors to check:  13048
0.99999994
1.0000001
0.99999994
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  10497
0.99999994
0.99999994
0.9999999
0.99999994
0.99999994
0.99999994
0.99999994
0.9999999
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  10159
0.99999994
1.0000001
1.0000001
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  12838
0.99999994
0.9999999
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
we have these many sectors to check:  13048
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
1.0000001
we have these many sectors to check:  13048
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
1.0000001
1.0000001
1.0000001
we have these many sectors t

In [185]:
arrayOfLumpes = []
indexesOfCandidates = [[] for i in range(12)]
for timeIndex in range(12):
    print(timeIndex)
    name = "/home/sepehr/thesis/APDataML/pickles/4normal_correlated_lumping_result_" + str(timeIndex) + ".pickle"
    with open(name, 'rb') as handle:
        b = pickle.load(handle)
    
    if len(b) > 0:
        minDegree = np.inf        
        for i in range(len(b)):
            if b[i][0] < minDegree:
                minDegree = b[i][0]

        minError = np.inf
        for i in range(len(b)):
            if b[i][0] == minDegree and b[i][1] < minError:
                minError = b[i][1]

        for i in range(len(b)):
            if b[i][0] == minDegree and b[i][1] == minError:
                indexesOfCandidates[timeIndex].append(b[i])

name = "/home/sepehr/thesis/APDataML/4normal_correlated_lumping_Boundaries.pickle"
with open(name, 'wb') as handle:
    pickle.dump(indexesOfCandidates, handle) 

0
1
2
3
4
5
6
7
8
9
10
11


In [187]:
b[0]

(1.0,
 48.4326171875,
 array([[4.00815910e-01, 1.73913043e-01, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00],
        [0.00000000e+00, 7.49239197e-01, 6.48548487e-03, 5.08130081e-04,
         0.00000000e+00],
        [0.00000000e+00, 3.01075269e-01, 3.52631579e-01, 3.75939850e-03,
         0.00000000e+00],
        [0.00000000e+00, 3.10344828e-01, 9.09090909e-02, 0.00000000e+00,
         0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00]]),
 [-1, 4, 21, 45, 69, 89])

# Lumping 15 Cons

## 9 Clusters

In [10]:
reload(oldLumping)

<module 'lumping_traditional' from '/home/sepehr/thesis/APDataML/lumping_traditional.py'>

In [11]:
for i in range(len(normalizedCuTrans_5min)):
    percentageMatrix_list, irreducible_matrix = preparingMatrixForLumping(normalizedCuTrans_5min[i])
    result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, False)
    nameResult = "/home/sepehr/thesis/APDataML/pickles/5normal_correlated_lumping_result_15Cons_" + str(i) + ".pickle"
    with open(nameResult, 'wb') as handle:
        pickle.dump(result, handle)
    namePercent = "/home/sepehr/thesis/APDataML/pickles/5normal_correlated_lumping_percentage_15Cons_" + str(i) + ".pickle"
    with open(namePercent, 'wb') as handle:
        pickle.dump(percentageMatrix_list, handle)

1.0000001
0.99999994
0.99999994
0.9999999
1.0000001
0.99999994
0.99999994
0.99999994
1.0000001
hello
we have these many sectors to check:  429768
0.99999994
1.0000001
0.99999994
1.0000001
1.0000001
1.0000001
hello
we have these many sectors to check:  108218
0.99999994
0.99999994
0.9999999
0.99999994
0.99999994
0.99999994
0.99999994
0.9999999
1.0000001
1.0000001
1.0000001
hello
we have these many sectors to check:  26495
0.99999994
1.0000001
1.0000001
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
hello
we have these many sectors to check:  358549
0.99999994
0.9999999
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
1.0000001
hello
we have these many sectors to check:  428575
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
1.0000001
hello
we have these many sectors to check:  429243
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
1.0000001
1.00000

In [234]:
result[10000]

(1.0,
 50.5382080078125,
 array([[4.65465465e-01, 3.04878049e-01, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [3.48432056e-04, 5.65853659e-01, 6.66666667e-03, 8.09716599e-04,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 1.51260504e-01, 5.05689001e-01, 1.35814889e-02,
         8.42815002e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 7.27272727e-02, 3.41463415e-01, 2.33532934e-01,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 8.33333333e-02, 8.33333333e-02, 1.33333333e-01,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e

## 8 Clusters

In [6]:
reload(oldLumping)

<module 'lumping_traditional' from '/home/sepehr/thesis/APDataML/lumping_traditional.py'>

In [8]:
for i in range(len(normalizedCuTrans_5min)):
    i = 2
    percentageMatrix_list, irreducible_matrix = preparingMatrixForLumping(normalizedCuTrans_5min[i])
    result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, False)
    nameResult = "/home/sepehr/thesis/APDataML/pickles/8normal_correlated_lumping_result_15Cons_" + str(i) + ".pickle"
    with open(nameResult, 'wb') as handle:
        pickle.dump(result, handle)
    namePercent = "/home/sepehr/thesis/APDataML/pickles/8normal_correlated_lumping_percentage_15Cons_" + str(i) + ".pickle"
    with open(namePercent, 'wb') as handle:
        pickle.dump(percentageMatrix_list, handle)
    break

0.99999994
0.99999994
0.9999999
0.99999994
0.99999994
0.99999994
0.99999994
0.9999999
1.0000001
1.0000001
1.0000001
hello
we have these many sectors to check:  2264796


In [337]:
def classifying(CU, boundaries):
    occupiedBandwidth = (CU / 255) * 100
    for i in range(len(boundaries)):
        if occupiedBandwidth <= boundaries[i]:
            return i
        
def dataPreparation(data, timeIndexes, minuteSplit, boundaries, steps):
    from sklearn.utils import shuffle
    warnings.filterwarnings('always')
    reg = "l2"
    solvers = "lbfgs"
    clf = LogisticRegression(penalty = reg, max_iter = 100000, random_state = 0,
                             solver = solvers , multi_class = 'multinomial')
    accuracyValue = 0
    numOfElements = 0
    f1scoreValue = 0
    precisionValue = 0
    recallValue = 0
    prevRowTrain = np.inf
    prevCU = np.inf

    sampleIntervals = 6 #seconds
    minuteSplit = 30 #minutes
    numOfSamples = minuteSplit * 60 / sampleIntervals
    # numberOfDays = len(numOfDays)
    days = np.zeros(7)
    numOfThirtyMinsPerDay = np.zeros(int((24 * 60) / minuteSplit)) #in this case 48
#     which6SecondsPerPeriod = np.zeros(int(minuteSplit * 60 / sampleIntervals)) #in this case 300
    prevRowTrain = np.inf
    prevCU = np.inf

    XArraysForLearning = []
    YArraysForLearning = []
    XArraysForTesting = []
    YArraysForTesting = []

    x = timeIndexes
    wholeDataFrame = data.loc[(data["timeIndex"] == x)].copy()
    
    wholeDataFrame["cuClass"] = wholeDataFrame["CU"].apply(lambda x: classifying(x, boundaries))

    

    stackCounter = 0
    prevCU = 0
    prev2CU = 0
    prev3CU = 0
    prev4CU = 0
    prev5CU = 0
    prevCUVal = 1
    print("start training set generation")
    #number of features are: prevCU + 48 correlations + 1 logDiff
    numberOfFeatures = 48 + 2*len(boundaries) + 2 + 10
    
    XArraysForLearning = np.zeros(shape=(len(wholeDataFrame), numberOfFeatures))
    YArraysForLearning = np.zeros(shape=(len(wholeDataFrame), len(boundaries)))
#     print(len(trainingDataFrame))
    counter = 0
    pandasIndexCounter = 0
    wholeDataFrame.reset_index()
    for index, row in wholeDataFrame.iterrows():
        if (index + steps) not in wholeDataFrame.index:
            continue
        booleanCont = 0
        for stationary in range(1, 48):
            if (index - stationary*steps) not in wholeDataFrame.index:
                booleanCont = 1
                continue
        if booleanCont == 1:
            continue
            
        XArraysForLearning[pandasIndexCounter, 0] = row["normalLogDiff"]
        for stationary in range(1, 48):
                XArraysForLearning[pandasIndexCounter, stationary] = wholeDataFrame.loc[index - stationary*steps]["normalLogDiff"]
                

        XArraysForLearning[pandasIndexCounter, 48 + wholeDataFrame.loc[index - steps]["cuClass"]] = 1
        XArraysForLearning[pandasIndexCounter, 48 + len(boundaries) + row["cuClass"]] = 1
        XArraysForLearning[pandasIndexCounter, 48 + 2 * len(boundaries)] = (wholeDataFrame.loc[index - steps]["CU"]/255)
        XArraysForLearning[pandasIndexCounter, 48 + 2 * len(boundaries) + 1] = (row["CU"] / 255)
        for corrs in range(10):
            XArraysForLearning[pandasIndexCounter, (48 + 2 * len(boundaries)) + 2 + corrs] = row["corr" + str(corrs + 1)]

#         diffLogValue = (np.log(row["CU"]) - np.log(wholeDataFrame.loc[index - steps]["CU"]))/ (np.log(255) - np.log(1))
        YArraysForLearning[pandasIndexCounter, wholeDataFrame.loc[index + steps]["cuClass"]] = 1
#         print(XArraysForLearning[pandasIndexCounter])

        which6SecondsPerPeriod = 0
    
        prevCU = row["cuClass"]
        prevCUVal = row["CU"]
        pandasIndexCounter += 1

    XArraysForLearning, YArraysForLearning = shuffle(XArraysForLearning, YArraysForLearning, random_state=0)
    XArraysForTraining = XArraysForLearning[:int(0.8*len(XArraysForLearning))]
    YArraysForTraining = YArraysForLearning[:int(0.8*len(YArraysForLearning))]
    XArraysForTesting = XArraysForLearning[int(0.8*len(XArraysForLearning)):]
    YArraysForTesting = YArraysForLearning[int(0.8*len(YArraysForLearning)):]
    print(XArraysForTraining.shape[0])
    print(XArraysForTesting.shape[0])


    return XArraysForTraining, YArraysForTraining, XArraysForTesting, YArraysForTesting, boundaries


def tensorFlowLossFunction(
    lossFuncBool, XArraysForLearning, YArraysForLearning, XArraysForTesting, YArraysForTesting, boundaries):
    
    CUDA_VISIBLE_DEVICES=""
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    os.environ['CUDA_VISIBLE_DEVICES'] = ''

    if tf.test.gpu_device_name():
        print('GPU found')
    else:
        print("No GPU found")
    
    batch_size = 64
    learning_rate = 0.01
    beta = 0.1
    numOfEpochs = 400
    graph = tf.Graph()
    with graph.as_default():
#         x = tf.placeholder(tf.float32, shape = (batch_size, XArraysForLearning.shape[1]))
#         y_ = tf.placeholder(tf.float32, shape = (batch_size, YArraysForLearning.shape[1]))
#         W = tf.Variable(tf.truncated_normal([XArraysForLearning.shape[1], YArraysForLearning.shape[1]], seed = 0), name="weights", dtype=tf.float32)
#         b = tf.Variable(tf.truncated_normal([YArraysForLearning.shape[1]], seed = 0), dtype=tf.float32)

        tf_test_dataset64 = tf.constant(XArraysForTesting)
        tf_test_dataset = tf.cast(tf_test_dataset64, tf.float32)

        
        x0 = tf.placeholder(tf.float32, shape = (batch_size, 48))
        x1 = tf.placeholder(tf.float32, shape = (batch_size, XArraysForLearning.shape[1] - 48))
        y0 = tf.placeholder(tf.float32, shape = (batch_size, XArraysForLearning.shape[1] - 47))
        y_ = tf.placeholder(tf.float32, shape = (batch_size, YArraysForLearning.shape[1]))
        W0 = tf.Variable(tf.truncated_normal([48, 1], seed = 0), name="weights0", dtype=tf.float32)
        W1 = tf.Variable(tf.truncated_normal([XArraysForLearning.shape[1] - 47, YArraysForLearning.shape[1]], seed = 0), name="weights1", dtype=tf.float32)
        b = tf.Variable(tf.truncated_normal([YArraysForLearning.shape[1]], seed = 0), dtype=tf.float32)
        print(x0)
        print(x1)
        print(W0)
        print(W1)
        
        result = tf.matmul(x0, W0)
#         print(y0)
        concated = tf.concat([result, x1], axis = 1)
#         print(concated)
        logits = tf.matmul(concated, W1)
        train_prediction = tf.nn.softmax(logits)
#         loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = y_)

        beta = 0.05
        
#         train_prediction = tf.nn.softmax(logits)
        # train_prediction = tf.nn.softmax_cross_entropy_with_logits_v2(labels = y_, logits = logits)
        testing = tf.concat([tf.matmul(tf_test_dataset[:,:48], W0), tf_test_dataset[:,48:]], axis = 1)
        test_prediction = tf.nn.softmax(tf.matmul(testing, W1))
        
#         logits = tf.matmul(x, W)

        # x = XArraysForLearning[0:(0 + batch_size), :]
        # y_ = tf.Variable(YArraysForLearning[0:(0 + batch_size), :])

        # loss = assymetricLossFunction(train_prediction, y_, boundaries)
        # loss = assymetricLossFunction(train_prediction, y_, boundaries)
        
        if lossFuncBool == 0:
            loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = y_)
        
        elif lossFuncBool == 1:
            loss = assymetricLossFunction(train_prediction, y_, boundaries)
#         dist = tfp.distributions.Multinomial(total_count=1, logits=logits)
#         loss = loss - (dist.log_prob(mult))
        # regularizer = tf.nn.l2_loss(W)
        # loss = tf.reduce_mean(loss + beta * regularizer)
        # loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits = train_prediction, labels = y_)
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

        prevAcc = 0
        prevLoss = np.inf

    with tf.Session(graph=graph) as session:
        initializer = tf.contrib.layers.xavier_initializer()
        tf.set_random_seed(0)
        W0 = tf.Variable(initializer([48, 1]))
        W1 = tf.Variable(initializer([XArraysForLearning.shape[1] - 47, YArraysForLearning.shape[1]]))
#         b = tf.Variable(initializer([YArraysForLearning.shape[1]]))       
        tf.global_variables_initializer().run()
#         print(W.eval())
        
        print("Initialized")

        numberOfBatchIteration = int(XArraysForLearning.shape[0] / batch_size)
        restOfData = XArraysForLearning.shape[0] % batch_size
        if restOfData != 0:
            numberOfBatchIteration += 1

        accuracy_result = 0
        accuracy_earlyStop = 0
        earlyStoppingCounter = 0
        
        for epoch in range(numOfEpochs):
            accuracyValue = 0
            lossValue = 0
            totalBatch = 0
            i = 0
            randomize = np.arange(XArraysForLearning.shape[0])
            random.Random(epoch).shuffle(randomize)
            XArraysForLearning = XArraysForLearning[randomize]
            YArraysForLearning = YArraysForLearning[randomize]

            for iteration in range(numberOfBatchIteration):
                if (iteration == numberOfBatchIteration - 1) and restOfData != 0:
                    break
                    batch_data = XArraysForLearning[i:, :]
                    batch_labels = YArraysForLearning[i:, :]

                else:
                    batch_data_x0 = XArraysForLearning[i:(i + batch_size), :48]
#                     print(batch_data_x0.shape)
                    batch_data_x1 = XArraysForLearning[i:(i + batch_size), 48:]
#                     print(batch_data_x1.shape)
                    batch_labels = YArraysForLearning[i:(i + batch_size), :]

                    i += batch_size


                feed_dict = {x0 : batch_data_x0, x1: batch_data_x1, y_ : batch_labels}
#                 print(y0.shape)
#                 print("aaaaaaaa")
#                 print(feed_dict)
                _, predictions, l = session.run([optimizer, train_prediction, loss], feed_dict=feed_dict)

#                 print(l)
                lossValue *= (iteration)
                lossValue += (np.sum(l))
                lossValue /= (iteration + 1)
                totalBatch += batch_size
#                     print(accuracy(batch_data, batch_labels))
                accuracyValue += accuracy(predictions, batch_labels) * batch_size
                
            testAcc = accuracy(test_prediction.eval(), YArraysForTesting)
            if testAcc < prevAcc and earlyStoppingCounter == 100:
                print(prevAcc)
                break

            elif testAcc < prevAcc:
                earlyStoppingCounter += 1

            elif testAcc >= prevAcc:
                prevAcc = testAcc
                earlyStoppingCounter = 0

            totalAccuracy = accuracyValue/totalBatch
#             print(session.run(W))
#             test_result = test_prediction.eval()
            if epoch % 20 == 0:
                print("epoch ", epoch, totalAccuracy, testAcc, lossValue)

#             print(accuracy(test_result, YArraysForTesting), prevAcc, earlyStoppingCounter)
                
            



        # print(session.run(W))
        predictionResult = test_prediction.eval()
        accResult = accuracy(predictionResult , YArraysForTesting)
        penaltyValue = assymetricPredictionScore(predictionResult, YArraysForTesting, boundaries)
#         print(W.eval())
        accuracy_result = prevAcc
        if accResult > prevAcc:
            accuracy_result = accResult
            
        correctLableIndex = np.argmax(YArraysForTesting, 1)
        predictionIndex = np.argmax(predictionResult, 1)
        
        precision = precision_score(correctLableIndex, predictionIndex, average='weighted')
        recall = recall_score(correctLableIndex, predictionIndex, average='weighted')
        f1Score = f1_score(correctLableIndex, predictionIndex, average='weighted')            
        
        return accuracy_result, penaltyValue, precision, recall, f1Score
            
    
def assymetricPredictionScore(predictedLables, trueLables, boundaries):
    xAxisPoints = np.linspace(rayleigh.ppf(0.01), rayleigh.ppf(0.99), 338)
    #number of overal datapoints must stay the same all the time
    maxState = 338

    inverseDistrib = max(rayleigh.pdf(xAxisPoints)) - rayleigh.pdf(xAxisPoints)
    minState = np.argmin(inverseDistrib)

    underUtilizedSum = 0
    overUtilizedSum = 0
    numberOfUnderUtilizedStates = minState - 0
    numberOfOverUtilizedStates = maxState - minState

    xAxisPoints -= xAxisPoints[np.argmin(inverseDistrib)]

    underUtilVal = numberOfUnderUtilizedStates / 100
    overUtilVal = numberOfOverUtilizedStates / 100

    correctLableIndex = np.argmax(trueLables, 1)
    predictionIndex = np.argmax(predictedLables, 1)

    diffPercentage = np.zeros(shape = (predictedLables.shape))

    penalties = np.zeros(shape = (predictedLables.shape))

    for index in range(predictedLables.shape[0]):
        diffPercentage[index] = boundaries[correctLableIndex[index]] - boundaries[:]


    for i in range(diffPercentage.shape[0]):
        for j in range(diffPercentage.shape[1]):
            if diffPercentage[i][j] > 0:
                penalties[i][j] = inverseDistrib[minState + math.floor
                                                  (diffPercentage[i][j] * overUtilVal)]
            else:
                penalties[i][j] = inverseDistrib[minState + math.floor
                                                  (diffPercentage[i][j] * underUtilVal)]

    sumOfPenalty = 0
    for i in range(predictionIndex.shape[0]):
        penalties[i] = penalties[i] / np.sum(penalties[i])
        sumOfPenalty += penalties[i][predictionIndex[i]]

    return sumOfPenalty
    
    
def accuracy(predictedLables, trueLables):
    import sys
    np.set_printoptions(threshold=sys.maxsize)
    correctLableIndex = np.argmax(trueLables, 1)
    predictionIndex = np.argmax(predictedLables, 1)
    errors = [0 for i in range(trueLables.shape[1])]
    corrects = [0 for i in range(trueLables.shape[1])]
    for i in range(len(correctLableIndex)):
        if correctLableIndex[i] != predictionIndex[i]:
            errors[correctLableIndex[i]] += 1
            corrects[predictionIndex[i]] += 1

    acc = np.float64(np.sum(correctLableIndex == predictionIndex)/predictedLables.shape[0])
    return acc
    
    
def assymetricLossFunction(prediction, correctLable, boundaries):
    xAxisPoints = np.linspace(rayleigh.ppf(0.01), rayleigh.ppf(0.99), 338)
    #number of overal datapoints must stay the same all the time
    maxState = 338
    inverseDistrib = max(rayleigh.pdf(xAxisPoints)) - rayleigh.pdf(xAxisPoints)
    inverseDistrib = tf.constant(inverseDistrib)
    xAxisPoints -= xAxisPoints[np.argmin(inverseDistrib)]
    minState = np.argmin(inverseDistrib)
    numberOfOverUtilizedStates = maxState - minState
    numberOfUnderUtilizedStates = minState
    minState = tf.constant(minState, tf.float32)
    numberOfOverUtilizedStates = tf.constant(numberOfOverUtilizedStates, tf.float32)
    numberOfUnderUtilizedStates = tf.constant(numberOfUnderUtilizedStates, tf.float32)

    underUtilVal = numberOfUnderUtilizedStates / 100
    overUtilVal = numberOfOverUtilizedStates / 100


    boundaries = tf.constant(boundaries, tf.float32)
    correctLableIndex = tf.argmax(correctLable, 1)

    diffPercentage = []
    for index in range(correctLableIndex.shape[0]):
        diffPercentage.append(boundaries[correctLableIndex[index]] - boundaries[:])

    diffPercentage = tf.stack(diffPercentage)

    penalties = []

    counter = 0
    for i in range(diffPercentage.shape[0]):
        for j in range(diffPercentage.shape[1]):
            counter += 1

            penalties.append(tf.cond(
                    tf.greater(diffPercentage[i][j], 0),
                    lambda: inverseDistrib[tf.dtypes.cast(minState + tf.math.floor
                                                                   (tf.math.scalar_mul(diffPercentage[i][j],
                                                                                       overUtilVal)), tf.int32)],
                    lambda: inverseDistrib[tf.dtypes.cast(minState + tf.math.floor
                                                                   (tf.math.scalar_mul
                                                                    (diffPercentage[i][j], underUtilVal))
                                                                   , tf.int32)]
                    ))

    penalties = tf.stack(penalties)
    penalties = tf.dtypes.cast(penalties, tf.float32)
    penalties = tf.reshape(penalties, diffPercentage.shape)
#     penalties = penalties / tf.norm(penalties)

    # weights = tf.reduce_sum(penalties * (1-prediction), axis=1)
    weights = (1 - penalties) * prediction
    weights = weights / tf.norm(weights)
    # print(correctLable)
    # print(prediction)
    # print(penalties)
    loss = tf.losses.softmax_cross_entropy(onehot_labels = correctLable, logits = weights)
    # weighted_losses = tf.nn.softmax_cross_entropy_with_logits_v2(labels = penalties, logits = prediction)
    # loss = tf.reduce_sum(weighted_losses)
    # loss = tf.reduce_sum(penalties * prediction)
    return tf.reduce_mean(loss)

In [333]:
percentageArray[0]

[[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 3.9215686274509802, True],
 [[10], 4.313725490196078, False],
 [[11], 4.705882352941177, True],
 [[12], 5.098039215686275, False],
 [[13, 14], 5.882352941176472, True],
 [[15], 6.27450980392157, False],
 [[16], 6.666666666666669, True],
 [[17], 7.058823529411767, False],
 [[18, 19], 7.843137254901964, True],
 [[20], 8.235294117647062, False],
 [[21], 8.62745098039216, True],
 [[22], 9.019607843137258, False],
 [[23, 24], 9.803921568627455, True],
 [[25], 10.196078431372554, False],
 [[26, 27], 10.98039215686275, True],
 [[28], 11.372549019607849, False],
 [[29], 11.764705882352947, True],
 [[30], 12.156862745098046, False],
 [[31, 32], 12.941176470588243, True],
 [[33], 13.333333333333341, False],
 [[34], 13.72549019607844, True],
 [[35], 14.117647058823538, False],
 [[36, 37], 14.901960784313735, True],
 [[38], 15.294117647058833, False],
 [[39], 15.686274509803932, True],
 [[40], 16.07843137254903, False],
 [[41, 42], 16.862745098039223, True],
 [[43

In [363]:
name = "/home/sepehr/thesis/APDataML/6merging_corrected_lumping_Boundaries.pickle"
with open(name, 'rb') as handle:
    normalLumping = pickle.load(handle)

In [364]:
percentageArray = []
for i in range(10):
    name = "/home/sepehr/thesis/APDataML/pickles/6merging_corrected_lumping_percentage_" + str(i) + ".pickle"
    with open(name, 'rb') as handle:
        percentageArray.append(pickle.load(handle))

In [338]:
accuracyResult = [np.inf for i in range(10)]
penaltyResult = [np.inf for i in range(10)]
precisionResult = [np.inf for i in range(10)]
recallResult = [np.inf for i in range(10)]
f1ScoreResult = [np.inf for i in range(10)]
boundariesResult = [np.inf for i in range(10)]
lumpAproxResult = [np.inf for i in range(10)]
lumpErrorResult = [np.inf for i in range(10)]


testDataFrame = {"accuracy": accuracyResult, "penalty": penaltyResult, "precision": precisionResult, 
                 "recall": recallResult, "f1Score": f1ScoreResult, "boundaries": boundariesResult, 
                 "lumpAprox": lumpAproxResult, "lumpError": lumpErrorResult}


resultDataFrameWithPenalty_5Class_merged = pd.DataFrame(testDataFrame)
resultDataFrameWithPenalty_5Class_merged = resultDataFrameWithPenalty_5Class_merged.astype('object')

In [340]:
warnings.filterwarnings('ignore')
for timeIndex in range(10):
    if len(normalLumping[timeIndex]) == 0:
        continue
    
    print(timeIndex)
    boundaries = np.array([])
    for i in range(1, len(normalLumping[0][0][3])):
        bound = normalLumping[timeIndex][0][3][i] + 1
        counter = 0
        for j in range(len(percentageArray[timeIndex])):
            if percentageArray[timeIndex][j][2] == False:
                counter += 1
            if counter == bound:
                boundaries = np.append(boundaries, percentageArray[timeIndex][j][1])
                break
    boundaries = np.append(boundaries,100)
    print(boundaries)
    
    XArraysForLearning, YArraysForLearning, XArraysForTesting, YArraysForTesting, boundaries = dataPreparation(
        data, timeIndex, 30, boundaries, 1)

    print("starting tensor")
    lossFunctionBoolean = 0
    accuracy_result, penaltyValue, precision, recall, f1Score = tensorFlowLossFunction(
        lossFunctionBoolean, XArraysForLearning, YArraysForLearning, XArraysForTesting, 
        YArraysForTesting, boundaries)
    
    resultDataFrameWithPenalty_5Class_merged.loc[timeIndex]["accuracy"] = accuracy_result
    resultDataFrameWithPenalty_5Class_merged.loc[timeIndex]["penalty"] = penaltyValue
    resultDataFrameWithPenalty_5Class_merged.loc[timeIndex]["precision"] = precision
    resultDataFrameWithPenalty_5Class_merged.loc[timeIndex]["recall"] = recall
    resultDataFrameWithPenalty_5Class_merged.loc[timeIndex]["f1Score"] = f1Score
    resultDataFrameWithPenalty_5Class_merged.loc[timeIndex]["boundaries"] = boundaries
    resultDataFrameWithPenalty_5Class_merged.loc[timeIndex]["lumpAprox"] = normalLumping[timeIndex][0][0]
    resultDataFrameWithPenalty_5Class_merged.loc[timeIndex]["lumpError"] = normalLumping[timeIndex][0][1]

    

0
[ 24.31372549  39.21568627  56.07843137  77.25490196  88.23529412
 100.        ]
start training set generation
252900
63226
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 24), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(25, 6) dtype=float32_ref>
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Initialized
epoch  0 0.9457336750189825 0.9468414892607472 10.26756537938173
epoch  20 0.9471771386990635 0.9470471008762218 9.002756403914173
epoch  40 0.9476675208807896 0.9468889380950875 8.96573752872368
epoch  60 0.9475567894203999 0.947790465947

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



1
[ 10.19607843  34.11764706  54.11764706  64.31372549  75.29411765
 100.        ]
start training set generation
130716
32679
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 24), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(25, 6) dtype=float32_ref>
Initialized
epoch  0 0.9400557051909892 0.9488356436855473 10.72681177870884
epoch  20 0.9504468658178257 0.9520181156094127 7.713012254553374
epoch  40 0.9509901444662096 0.9529667370482573 7.615481737486647
epoch  60 0.9510819662095984 0.9520181156094127 7.580492972568714
epoch  80 0.9510590107737512 0.94993726858227 7.558256167097492
epoch  100 0.9507605901077375 0.951344900394749 7.542806316542931
epoch  120 0.9513191723800196 0.9521711190672909 7.529225627834252
epoch  140 0.9510972698334965 0.951834511459959 7.5186807702675855
epoch  160 0.9508447600391773 0.9515285045442027 7.510390930264517
epoch  18

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



4
[ 11.37254902  36.07843137  51.37254902  67.05882353  81.17647059
 100.        ]
start training set generation
69260
17316
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 24), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(25, 6) dtype=float32_ref>
Initialized
epoch  0 0.8940908040665434 0.9232501732501732 22.917856337843446
epoch  20 0.9262505776340111 0.9263686763686764 14.085248570785946
epoch  40 0.9266404805914972 0.926946176946177 13.98377851968332
epoch  60 0.9267848890942699 0.9270616770616771 13.950544552529808
epoch  80 0.9276224584103512 0.9266574266574267 13.926475094999711
epoch  100 0.9267126848428835 0.9257911757911758 13.905537801837745
epoch  120 0.9273769639556377 0.9263686763686764 13.929191817637951
epoch  140 0.9273192005545287 0.9273504273504274 13.905074169807646
epoch  160 0.9270159426987061 0.9267151767151767 13.909707099364557


/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



8
[ 11.37254902  36.07843137  60.39215686  78.03921569  89.01960784
 100.        ]
start training set generation
31791
7948
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 24), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(25, 6) dtype=float32_ref>
Initialized
epoch  0 0.8686050907258065 0.9097886260694514 27.810016033149534
epoch  20 0.9168031754032258 0.9106693507800705 13.916302437263152
epoch  40 0.9174017137096774 0.912179164569703 13.697595060833041
epoch  60 0.9182207661290323 0.9116758933064922 13.661370178384166
epoch  80 0.9164881552419355 0.9125566180171112 13.578466052970578
epoch  100 0.9172127016129032 0.9107951685958732 13.593522375149112
epoch  120 0.9179057459677419 0.9096628082536488 13.545851731973311
epoch  140 0.9169291834677419 0.9102918973326624 13.495725581722871
epoch  160 0.9173387096774194 0.9094111726220433 13.50266111762293
0

In [359]:
# resultDataFrameNoPenalty_5Class_merged = resultDataFrameWithPenalty_5Class_merged.copy()

In [377]:
resultDataFrameNoPenalty_5Class_merged.to_pickle("resultDataFrameNoPenalty_5Class_merged.pickle")

In [376]:
resultDataFrameNoPenalty_5Class_merged

,accuracy,penalty,precision,recall,f1Score,boundaries,lumpAprox,lumpError
0,0.948328,393.319,0.933715,0.946636,0.937693,"[24.31372549019606, 39.215686274509736, 56.078...",1,42.1122
1,0.953365,173.36,0.949209,0.952385,0.949681,"[10.196078431372554, 34.11764705882348, 54.117...",1,49.8154
2,0.946461,66.3753,0.93482,0.944944,0.934911,"[10.196078431372554, 33.333333333333286, 52.15...",1,39.2705
3,0.92786,82.7095,0.917224,0.926335,0.91771,"[10.196078431372554, 28.235294117647026, 45.09...",1,33.2637
4,0.92787,149.207,0.908067,0.926773,0.911105,"[11.372549019607849, 36.07843137254896, 51.372...",1,39.7256
5,0.956863,7.56439,0.943396,0.952288,0.945904,"[12.156862745098046, 36.07843137254896, 46.274...",1,36.2256
6,0.859932,679.631,0.839629,0.857086,0.842906,"[13.333333333333341, 38.039215686274446, 52.15...",0.628418,35.7321
7,0.834189,291.779,0.826881,0.832166,0.825594,"[12.156862745098046, 29.01960784313722, 53.333...",0.727539,40.8757
8,0.914066,96.0214,0.903558,0.911928,0.90451,"[11.372549019607849, 36.07843137254896, 60.392...",1,38.6326
9,0.893023,478.771,0.877248,0.890723,0.880292,"[13.333333333333341, 38.039215686274446, 54.11...",0.539551,37.42


In [ ]:
warnings.filterwarnings('ignore')
for timeIndex in range(10):
    if len(normalLumping[timeIndex]) == 0:
        continue
    
    print(timeIndex)
    boundaries = np.array([])
    for i in range(1, len(normalLumping[0][0][3])):
        bound = normalLumping[timeIndex][0][3][i] + 1
        counter = 0
        for j in range(len(percentageArray[timeIndex])):
            if percentageArray[timeIndex][j][2] == False:
                counter += 1
            if counter == bound:
                boundaries = np.append(boundaries, percentageArray[timeIndex][j][1])
                break
    boundaries = np.append(boundaries,100)
    print(boundaries)
    
    XArraysForLearning, YArraysForLearning, XArraysForTesting, YArraysForTesting, boundaries = dataPreparation(
        data, timeIndex, 30, boundaries, 1)

    print("starting tensor")
    lossFunctionBoolean = 1
    accuracy_result, penaltyValue, precision, recall, f1Score = tensorFlowLossFunction(
        lossFunctionBoolean, XArraysForLearning, YArraysForLearning, XArraysForTesting, 
        YArraysForTesting, boundaries)
    
    resultDataFrameWithPenalty_5Class_merged.loc[timeIndex]["accuracy"] = accuracy_result
    resultDataFrameWithPenalty_5Class_merged.loc[timeIndex]["penalty"] = penaltyValue
    resultDataFrameWithPenalty_5Class_merged.loc[timeIndex]["precision"] = precision
    resultDataFrameWithPenalty_5Class_merged.loc[timeIndex]["recall"] = recall
    resultDataFrameWithPenalty_5Class_merged.loc[timeIndex]["f1Score"] = f1Score
    resultDataFrameWithPenalty_5Class_merged.loc[timeIndex]["boundaries"] = boundaries
    resultDataFrameWithPenalty_5Class_merged.loc[timeIndex]["lumpAprox"] = normalLumping[timeIndex][0][0]
    resultDataFrameWithPenalty_5Class_merged.loc[timeIndex]["lumpError"] = normalLumping[timeIndex][0][1]

0
[ 24.31372549  39.21568627  56.07843137  77.25490196  88.23529412
 100.        ]
start training set generation


In [373]:
resultDataFrameWithPenalty_5Class_merged

,accuracy,penalty,precision,recall,f1Score,boundaries,lumpAprox,lumpError
0,0.948297,425.337,0.930687,0.94545,0.936816,"[24.31372549019606, 39.215686274509736, 56.078...",1,42.1122
1,0.952722,149.707,0.949481,0.951314,0.949175,"[10.196078431372554, 34.11764705882348, 54.117...",1,49.8154
2,0.944511,55.0486,0.935454,0.942126,0.937824,"[10.196078431372554, 33.333333333333286, 52.15...",1,39.2705
3,0.9214,81.595,0.894106,0.920592,0.905278,"[10.196078431372554, 28.235294117647026, 45.09...",1,33.2637
4,0.927639,131.566,0.913317,0.926484,0.913408,"[11.372549019607849, 36.07843137254896, 51.372...",1,39.7256
5,0.948366,5.71252,0.940381,0.947059,0.938722,"[12.156862745098046, 36.07843137254896, 46.274...",1,36.2256
6,0.857108,569.217,0.842419,0.852506,0.843982,"[13.333333333333341, 38.039215686274446, 52.15...",0.628418,35.7321
7,0.829521,245.019,0.821042,0.825319,0.82242,"[12.156862745098046, 29.01960784313722, 53.333...",0.727539,40.8757
8,0.911928,81.0408,0.904607,0.90463,0.904134,"[11.372549019607849, 36.07843137254896, 60.392...",1,38.6326
9,0.892095,361.219,0.876286,0.886977,0.877343,"[13.333333333333341, 38.039215686274446, 54.11...",0.539551,37.42


In [378]:
resultDataFrameWithPenalty_5Class_merged.to_pickle("resultDataFrameWithPenalty_5Class_merged.pickle")

In [73]:
resultDataFrameWithPenalty_5Class_merged

,accuracy,penalty,precision,recall,f1Score,boundaries,lumpAprox,lumpError
0,0.936703,795.811,0.925707,0.929019,0.927247,"[24.31372549019606, 39.215686274509736, 56.078...",1,42.0209
1,0.965442,33.6358,0.951433,0.963409,0.956896,"[10.196078431372554, 33.333333333333286, 54.11...",1,46.7483
2,0.940062,49.0424,0.93037,0.938102,0.933027,"[11.372549019607849, 36.07843137254896, 56.078...",1,35.325
3,0.92282,142.954,0.914871,0.922365,0.91532,"[13.333333333333341, 38.039215686274446, 62.35...",1,39.6284
4,0.924218,183.717,0.90891,0.922562,0.908958,"[11.372549019607849, 36.07843137254896, 59.215...",1,41.1243
5,0.873706,624.08,0.865549,0.87101,0.863595,"[15.294117647058833, 39.215686274509736, 63.13...",0.666992,36.5415
6,0.899134,78.7307,0.893219,0.893835,0.893076,"[13.333333333333341, 38.039215686274446, 60.39...",0.75,41.4802
7,0.89477,24.4609,0.895363,0.889166,0.888735,"[23.13725490196077, 47.05882352941167, 71.3725...",1,50.7913
8,0.867637,211.883,0.856573,0.864156,0.854237,"[11.372549019607849, 35.29411764705877, 58.039...",0.777832,38.2456
9,0.917732,63.5224,0.907165,0.915032,0.907276,"[11.372549019607849, 36.07843137254896, 57.254...",1,46.4207


In [ ]:
name = "/home/sepehr/thesis/APDataML/7merging_corrected_lumping_Boundaries.pickle"
with open(name, 'rb') as handle:
    normalLumping = pickle.load(handle)

In [ ]:
percentageArray = []
for i in range(10):
    name = "/home/sepehr/thesis/APDataML/pickles/7merging_corrected_lumping_percentage_" + str(i) + ".pickle"
    with open(name, 'rb') as handle:
        percentageArray.append(pickle.load(handle))

In [344]:
accuracyResult = [np.inf for i in range(10)]
penaltyResult = [np.inf for i in range(10)]
precisionResult = [np.inf for i in range(10)]
recallResult = [np.inf for i in range(10)]
f1ScoreResult = [np.inf for i in range(10)]
boundariesResult = [np.inf for i in range(10)]
lumpAproxResult = [np.inf for i in range(10)]
lumpErrorResult = [np.inf for i in range(10)]


testDataFrame = {"accuracy": accuracyResult, "penalty": penaltyResult, "precision": precisionResult, 
                 "recall": recallResult, "f1Score": f1ScoreResult, "boundaries": boundariesResult, 
                 "lumpAprox": lumpAproxResult, "lumpError": lumpErrorResult}


resultDataFrameNoPenalty_6Class_merged = pd.DataFrame(testDataFrame)
resultDataFrameNoPenalty_6Class_merged = resultDataFrameNoPenalty_6Class_merged.astype('object')

In [346]:
warnings.filterwarnings('ignore')
for timeIndex in range(10):
    if len(normalLumping[timeIndex]) == 0:
        continue
    
    print(timeIndex)
    boundaries = np.array([])
    for i in range(1, len(normalLumping[0][0][3])):
        bound = normalLumping[timeIndex][0][3][i] + 1
        counter = 0
        for j in range(len(percentageArray[timeIndex])):
            if percentageArray[timeIndex][j][2] == False:
                counter += 1
            if counter == bound:
                boundaries = np.append(boundaries, percentageArray[timeIndex][j][1])
                break
    boundaries = np.append(boundaries,100)
    print(boundaries)
    
    XArraysForLearning, YArraysForLearning, XArraysForTesting, YArraysForTesting, boundaries = dataPreparation(
        data, timeIndex, 30, boundaries, 1)

    print("starting tensor")
    lossFunctionBoolean = 0
    accuracy_result, penaltyValue, precision, recall, f1Score = tensorFlowLossFunction(
        lossFunctionBoolean, XArraysForLearning, YArraysForLearning, XArraysForTesting, 
        YArraysForTesting, boundaries)
    
    resultDataFrameNoPenalty_6Class_merged.loc[timeIndex]["accuracy"] = accuracy_result
    resultDataFrameNoPenalty_6Class_merged.loc[timeIndex]["penalty"] = penaltyValue
    resultDataFrameNoPenalty_6Class_merged.loc[timeIndex]["precision"] = precision
    resultDataFrameNoPenalty_6Class_merged.loc[timeIndex]["recall"] = recall
    resultDataFrameNoPenalty_6Class_merged.loc[timeIndex]["f1Score"] = f1Score
    resultDataFrameNoPenalty_6Class_merged.loc[timeIndex]["boundaries"] = boundaries
    resultDataFrameNoPenalty_6Class_merged.loc[timeIndex]["lumpAprox"] = normalLumping[timeIndex][0][0]
    resultDataFrameNoPenalty_6Class_merged.loc[timeIndex]["lumpError"] = normalLumping[timeIndex][0][1]

    

0
[ 10.19607843  32.15686275  43.1372549   56.07843137  77.25490196
  88.23529412 100.        ]
start training set generation
252900
63226
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 26), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(27, 7) dtype=float32_ref>
Initialized
epoch  0 0.9260353391546444 0.92953848100465 12.576420962010955
epoch  20 0.9330549228043533 0.932053269224686 10.649581848986694
epoch  40 0.9341899202733486 0.9329864296333786 10.563785796677186
epoch  60 0.933917046317388 0.9333343877518743 10.540652280818115
epoch  80 0.9337390850417616 0.9328124505741309 10.525521206964378
epoch  100 0.9340159136927361 0.9335874482016892 10.503396468233724
epoch  120 0.9339051822323462 0.930740518141271 10.50198771642207
epoch  140 0.9339882308276386 0.9320216366684592 10.49153361022397
epoch  160 0.9339407744874715 0.9319741878341189 10.4803554

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



1
[ 10.19607843  32.15686275  49.01960784  59.21568627  73.33333333
  84.31372549 100.        ]
start training set generation
130716
32679
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 26), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(27, 7) dtype=float32_ref>
Initialized
epoch  0 0.940583680215475 0.9447045503228373 10.353486644707987
epoch  20 0.9473096229187071 0.9478258208635515 8.256434939400346
epoch  40 0.9476156953966699 0.947948223629854 8.152467164657011
epoch  60 0.9476386508325172 0.9484072340034885 8.103151579269817
epoch  80 0.9476769098922625 0.9463875883594969 8.081095460287871
epoch  100 0.9477687316356513 0.9480400257045809 8.070826406343407
epoch  120 0.9483885284035259 0.9486214388445179 8.04490886723495
epoch  140 0.948174277668952 0.9480400257045809 8.040674098500082
epoch  160 0.9477993388834476 0.9483460326203372 8.036504835561

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



2
[ 10.19607843  34.11764706  46.2745098   61.17647059  71.37254902
  82.35294118 100.        ]
start training set generation
36906
9227
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 26), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(27, 7) dtype=float32_ref>
Initialized
epoch  0 0.92626953125 0.9435352769047362 16.022556380265257
epoch  20 0.9419759114583334 0.9439687872547957 10.440113161173132
epoch  40 0.9414876302083334 0.944835807954915 10.363766828137964
epoch  60 0.9427625868055556 0.9455944510675193 10.322150641431401
epoch  80 0.9436306423611112 0.9455944510675193 10.291224640276697
epoch  100 0.9439290364583334 0.9460279614175788 10.247130079608826
epoch  120 0.9442816840277778 0.9453776958924894 10.218273385738332
epoch  140 0.9440104166666666 0.9463530941801236 10.215138794233399
epoch  160 0.9442003038194444 0.945811206242549 10.205541654

...

4
[ 11.37254902  36.07843137  47.05882353  59.21568627  70.19607843
  81.17647059 100.        ]
start training set generation
69260
17316
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 26), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(27, 7) dtype=float32_ref>
Initialized
epoch  0 0.9101634704251387 0.9230769230769231 19.604902251150165
epoch  20 0.9255429759704251 0.9261954261954262 14.866226706178702
epoch  40 0.9258173521256932 0.9270616770616771 14.750451625825738
epoch  60 0.9259617606284658 0.9271194271194271 14.715622816861448
epoch  80 0.9266260397412199 0.9267151767151767 14.680675843727123
epoch  100 0.925918438077634 0.9263686763686764 14.677527591393309
epoch  120 0.9264816312384473 0.9268884268884269 14.689059328019287
epoch  140 0.9261206099815157 0.9272349272349273 14.673012304438242
epoch  160 0.9264960720887245 0.9262531762531763 14.65

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



start training set generation
179911
44978
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 26), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(27, 7) dtype=float32_ref>
Initialized
epoch  0 0.7407728566346496 0.7458757614833919 43.401287602682984
epoch  20 0.7533851387406617 0.7510783049490862 40.12914467265029
epoch  40 0.753863171469228 0.7520120948019031 40.09165750080805
epoch  60 0.7541577730345073 0.7509004402152163 40.04372837211857
epoch  80 0.7540132515119174 0.7492774245186535 40.039571156157486
epoch  100 0.754730300604767 0.7519231624349683 40.021351671439355
epoch  120 0.7544634916399857 0.7504557783805416 40.0239506629557
epoch  140 0.7536686232657417 0.7492329583351861 40.01963491042887
0.7520120948019031
7
[ 12.15686275  29.01960784  53.33333333  66.2745098   77.25490196
  88.23529412 100.        ]
start training set generation
51405
12852

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



8
[ 11.37254902  36.07843137  49.01960784  61.17647059  78.03921569
  89.01960784 100.        ]
start training set generation
31791
7948
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 26), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(27, 7) dtype=float32_ref>
Initialized
epoch  0 0.8937121975806451 0.9070206341217917 21.566528465478658
epoch  20 0.9140625 0.9080271766482134 15.253708775966398
epoch  40 0.9146295362903226 0.9094111726220433 15.015051963829226
epoch  60 0.9155430947580645 0.9091595369904378 14.977333666816834
epoch  80 0.9142515120967742 0.9105435329642677 14.890992243443765
epoch  100 0.9151650705645161 0.9091595369904378 14.897398262254653
epoch  120 0.9158266129032258 0.9091595369904378 14.861562356833488
epoch  140 0.9148500504032258 0.9089079013588324 14.80419241372616
epoch  160 0.9147240423387096 0.9079013588324106 14.817655131701

In [ ]:
# resultDataFrameWithPenalty_6Class_merged = resultDataFrameNoPenalty_6Class_merged.copy()

In [374]:
resultDataFrameNoPenalty_6Class_merged

,accuracy,penalty,precision,recall,f1Score,boundaries,lumpAprox,lumpError
0,0.933841,339.037,0.924615,0.933113,0.926657,"[10.196078431372554, 32.156862745097996, 43.13...",1,42.3945
1,0.949692,162.243,0.944154,0.947765,0.945398,"[10.196078431372554, 32.156862745097996, 49.01...",1,49.2557
2,0.948304,51.3612,0.933489,0.944727,0.936418,"[10.196078431372554, 34.11764705882348, 46.274...",1,39.2212
3,0.9441,57.2209,0.932674,0.943383,0.934844,"[10.196078431372554, 33.333333333333286, 46.27...",1,34.7075
4,0.927755,125.415,0.908417,0.926831,0.910832,"[11.372549019607849, 36.07843137254896, 47.058...",1,39.0748
5,0.955556,6.39329,0.942787,0.951634,0.946349,"[12.156862745098046, 36.07843137254896, 46.274...",1,37.8953
6,0.752012,820.873,0.727508,0.7501,0.729844,"[12.156862745098046, 29.01960784313722, 39.215...",0.628418,36.2811
7,0.834578,297.744,0.823769,0.830999,0.821401,"[12.156862745098046, 29.01960784313722, 53.333...",0.727539,39.1799
8,0.912557,62.8686,0.896603,0.910795,0.897075,"[11.372549019607849, 36.07843137254896, 49.019...",1,37.4873
9,0.83946,483.93,0.820383,0.838941,0.825786,"[23.13725490196077, 39.215686274509736, 50.196...",0.590332,36.8445


In [379]:
resultDataFrameNoPenalty_6Class_merged.to_pickle("resultDataFrameNoPenalty_6Class_merged.pickle")

In [ ]:
warnings.filterwarnings('ignore')
for timeIndex in range(10):
    if len(normalLumping[timeIndex]) == 0:
        continue
    
    print(timeIndex)
    boundaries = np.array([])
    for i in range(1, len(normalLumping[0][0][3])):
        bound = normalLumping[timeIndex][0][3][i] + 1
        counter = 0
        for j in range(len(percentageArray[timeIndex])):
            if percentageArray[timeIndex][j][2] == False:
                counter += 1
            if counter == bound:
                boundaries = np.append(boundaries, percentageArray[timeIndex][j][1])
                break
    boundaries = np.append(boundaries,100)
    print(boundaries)
    
    XArraysForLearning, YArraysForLearning, XArraysForTesting, YArraysForTesting, boundaries = dataPreparation(
        data, timeIndex, 30, boundaries, 1)

    print("starting tensor")
    lossFunctionBoolean = 1
    accuracy_result, penaltyValue, precision, recall, f1Score = tensorFlowLossFunction(
        lossFunctionBoolean, XArraysForLearning, YArraysForLearning, XArraysForTesting, 
        YArraysForTesting, boundaries)
    
    resultDataFrameWithPenalty_6Class_merged.loc[timeIndex]["accuracy"] = accuracy_result
    resultDataFrameWithPenalty_6Class_merged.loc[timeIndex]["penalty"] = penaltyValue
    resultDataFrameWithPenalty_6Class_merged.loc[timeIndex]["precision"] = precision
    resultDataFrameWithPenalty_6Class_merged.loc[timeIndex]["recall"] = recall
    resultDataFrameWithPenalty_6Class_merged.loc[timeIndex]["f1Score"] = f1Score
    resultDataFrameWithPenalty_6Class_merged.loc[timeIndex]["boundaries"] = boundaries
    resultDataFrameWithPenalty_6Class_merged.loc[timeIndex]["lumpAprox"] = normalLumping[timeIndex][0][0]
    resultDataFrameWithPenalty_6Class_merged.loc[timeIndex]["lumpError"] = normalLumping[timeIndex][0][1]

    

In [375]:
resultDataFrameWithPenalty_6Class_merged

,accuracy,penalty,precision,recall,f1Score,boundaries,lumpAprox,lumpError
0,0.933841,339.037,0.924615,0.933113,0.926657,"[10.196078431372554, 32.156862745097996, 43.13...",1,42.3945
1,0.949692,162.243,0.944154,0.947765,0.945398,"[10.196078431372554, 32.156862745097996, 49.01...",1,49.2557
2,0.948304,51.3612,0.933489,0.944727,0.936418,"[10.196078431372554, 34.11764705882348, 46.274...",1,39.2212
3,0.9441,57.2209,0.932674,0.943383,0.934844,"[10.196078431372554, 33.333333333333286, 46.27...",1,34.7075
4,0.927755,125.415,0.908417,0.926831,0.910832,"[11.372549019607849, 36.07843137254896, 47.058...",1,39.0748
5,0.955556,6.39329,0.942787,0.951634,0.946349,"[12.156862745098046, 36.07843137254896, 46.274...",1,37.8953
6,0.752012,820.873,0.727508,0.7501,0.729844,"[12.156862745098046, 29.01960784313722, 39.215...",0.628418,36.2811
7,0.834578,297.744,0.823769,0.830999,0.821401,"[12.156862745098046, 29.01960784313722, 53.333...",0.727539,39.1799
8,0.912557,62.8686,0.896603,0.910795,0.897075,"[11.372549019607849, 36.07843137254896, 49.019...",1,37.4873
9,0.83946,483.93,0.820383,0.838941,0.825786,"[23.13725490196077, 39.215686274509736, 50.196...",0.590332,36.8445


In [354]:
name = "/home/sepehr/thesis/APDataML/5merging_corrected_lumping_Boundaries.pickle"
with open(name, 'rb') as handle:
    normalLumping = pickle.load(handle)

In [355]:
percentageArray = []
for i in range(10):
    name = "/home/sepehr/thesis/APDataML/pickles/5merging_corrected_lumping_percentage_" + str(i) + ".pickle"
    with open(name, 'rb') as handle:
        percentageArray.append(pickle.load(handle))

In [356]:
accuracyResult = [np.inf for i in range(10)]
penaltyResult = [np.inf for i in range(10)]
precisionResult = [np.inf for i in range(10)]
recallResult = [np.inf for i in range(10)]
f1ScoreResult = [np.inf for i in range(10)]
boundariesResult = [np.inf for i in range(10)]
lumpAproxResult = [np.inf for i in range(10)]
lumpErrorResult = [np.inf for i in range(10)]


testDataFrame = {"accuracy": accuracyResult, "penalty": penaltyResult, "precision": precisionResult, 
                 "recall": recallResult, "f1Score": f1ScoreResult, "boundaries": boundariesResult, 
                 "lumpAprox": lumpAproxResult, "lumpError": lumpErrorResult}


resultDataFrameNoPenalty_4Class_merged = pd.DataFrame(testDataFrame)
resultDataFrameNoPenalty_4Class_merged = resultDataFrameNoPenalty_4Class_merged.astype('object')

In [357]:
warnings.filterwarnings('ignore')
for timeIndex in range(10):
    if len(normalLumping[timeIndex]) == 0:
        continue
    
    print(timeIndex)
    boundaries = np.array([])
    for i in range(1, len(normalLumping[0][0][3])):
        bound = normalLumping[timeIndex][0][3][i] + 1
        counter = 0
        for j in range(len(percentageArray[timeIndex])):
            if percentageArray[timeIndex][j][2] == False:
                counter += 1
            if counter == bound:
                boundaries = np.append(boundaries, percentageArray[timeIndex][j][1])
                break
    boundaries = np.append(boundaries,100)
    print(boundaries)
    
    XArraysForLearning, YArraysForLearning, XArraysForTesting, YArraysForTesting, boundaries = dataPreparation(
        data, timeIndex, 30, boundaries, 1)

    print("starting tensor")
    lossFunctionBoolean = 0
    accuracy_result, penaltyValue, precision, recall, f1Score = tensorFlowLossFunction(
        lossFunctionBoolean, XArraysForLearning, YArraysForLearning, XArraysForTesting, 
        YArraysForTesting, boundaries)
    
    resultDataFrameNoPenalty_4Class_merged.loc[timeIndex]["accuracy"] = accuracy_result
    resultDataFrameNoPenalty_4Class_merged.loc[timeIndex]["penalty"] = penaltyValue
    resultDataFrameNoPenalty_4Class_merged.loc[timeIndex]["precision"] = precision
    resultDataFrameNoPenalty_4Class_merged.loc[timeIndex]["recall"] = recall
    resultDataFrameNoPenalty_4Class_merged.loc[timeIndex]["f1Score"] = f1Score
    resultDataFrameNoPenalty_4Class_merged.loc[timeIndex]["boundaries"] = boundaries
    resultDataFrameNoPenalty_4Class_merged.loc[timeIndex]["lumpAprox"] = normalLumping[timeIndex][0][0]
    resultDataFrameNoPenalty_4Class_merged.loc[timeIndex]["lumpError"] = normalLumping[timeIndex][0][1]

    

0
[ 24.31372549  39.21568627  56.07843137  77.25490196 100.        ]
start training set generation
252900
63226
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 22), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(23, 5) dtype=float32_ref>
Initialized
epoch  0 0.9418580739053404 0.9467465915920665 11.227197784135628
epoch  20 0.9471890027841053 0.947015468319995 8.996241589462672
epoch  40 0.9475844722854974 0.94676240787018 8.958842988105935
epoch  60 0.9474776955201215 0.9478062822256667 8.947843407202708
epoch  80 0.9475211971652746 0.9464144497516844 8.938231769376909
epoch  100 0.9474183750949127 0.9468256729826338 8.933609255957517
epoch  120 0.9472601872943559 0.9457027172365798 8.932360232615576
epoch  140 0.9473669640597318 0.9475848543320786 8.920036000319593
epoch  160 0.9475726082004556 0.9465884288109322 8.928873050577941
epoch  180 0.947643792

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



1
[ 10.19607843  34.11764706  54.11764706  75.29411765 100.        ]
start training set generation
130716
32679
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 22), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(23, 5) dtype=float32_ref>
Initialized
epoch  0 0.9447386141038198 0.948866244377123 9.614014570673355
epoch  20 0.9505463393731636 0.9520487163009884 7.678841442424333
epoch  40 0.9509289299706171 0.9528137335903791 7.589706227314692
epoch  60 0.9511661361410382 0.9519263135346859 7.558815839533474
epoch  80 0.9510590107737512 0.9502126748064507 7.53811769404911
epoch  100 0.9508753672869735 0.9514367024694758 7.525329636605084
epoch  120 0.951250306072478 0.9521099176841397 7.5118020903118445
epoch  140 0.9511890915768854 0.9519263135346859 7.5004855682170355
epoch  160 0.9508218046033301 0.9514367024694758 7.493789761995825
epoch  180 0.95137273

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



2
[ 10.19607843  34.11764706  58.03921569  80.39215686 100.        ]
start training set generation
36906
9227
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 22), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(23, 5) dtype=float32_ref>
Initialized
epoch  0 0.9302571614583334 0.9474368700552726 13.633635831375909
epoch  20 0.9443359375 0.948087135580362 9.275190155539244
epoch  40 0.9449055989583334 0.949170911455511 9.178620128995837
epoch  60 0.9451768663194444 0.9500379321556303 9.14893312979904
epoch  80 0.9456651475694444 0.9498211769806004 9.12503265817131
epoch  100 0.9459906684027778 0.9496044218055706 9.112990272955763
epoch  120 0.9464246961805556 0.9501463097431451 9.091652866452925
epoch  140 0.9459635416666666 0.9497127993930855 9.094044027850046
epoch  160 0.9467230902777778 0.9503630649181749 9.086728773183289
epoch  180 0.9461263020833334 0

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



4
[ 11.37254902  36.07843137  58.03921569  82.35294118 100.        ]
start training set generation
69260
17316
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 22), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(23, 5) dtype=float32_ref>
Initialized
epoch  0 0.9140336182994455 0.9256179256179257 16.83594924773397
epoch  20 0.9270737060998152 0.9278701778701779 13.226627999885682
epoch  40 0.9275358133086876 0.9282744282744283 13.142763008471992
epoch  60 0.9276224584103512 0.9286786786786787 13.112824478784015
epoch  80 0.9280845656192237 0.9283321783321783 13.090966235026857
epoch  100 0.927391404805915 0.9256179256179257 13.08553476342432
0.9294294294294294
5
[ 19.21568627  43.1372549   63.1372549   79.21568627 100.        ]
start training set generation
6117
1530
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor(

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



8
[ 11.37254902  36.07843137  60.39215686  78.03921569 100.        ]
start training set generation
31791
7948
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 22), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(23, 5) dtype=float32_ref>
Initialized
epoch  0 0.8959803427419355 0.909536990437846 19.75873961564033
epoch  20 0.9163621471774194 0.9107951685958732 13.859031512852638
epoch  40 0.9172127016129032 0.9111726220432813 13.678526803370444
epoch  60 0.918000252016129 0.9112984398590841 13.65998017980206
epoch  80 0.9162361391129032 0.9123049823855058 13.58205785578297
epoch  100 0.916960685483871 0.9112984398590841 13.604036635448859
epoch  120 0.9176852318548387 0.9092853548062405 13.559295207262043
epoch  140 0.9165826612903226 0.9094111726220433 13.511940397562515
epoch  160 0.9174332157258065 0.909536990437846 13.519515465344156
0.9133115249119276
9

In [381]:
resultDataFrameNoPenalty_4Class_merged.to_pickle("resultDataFrameNoPenalty_4Class_merged.pickle")

In [91]:
samples = [93828, 6887, 6123, 13177, 14489, 34491, 5661, 1606, 11204, 5555, 19788, 15051]
np.sum(resultDataFrameNoPenalty_5Class_merged["accuracy"] * samples)/np.sum(samples)

0.9195427016589134

In [88]:
resultDataFrameWithPenalty_5Class_merged

,accuracy,penalty,precision,recall,f1Score,boundaries,lumpAprox,lumpError
0,0.936703,795.811,0.925707,0.929019,0.927247,"[24.31372549019606, 39.215686274509736, 56.078...",1,42.0209
1,0.965442,33.6358,0.951433,0.963409,0.956896,"[10.196078431372554, 33.333333333333286, 54.11...",1,46.7483
2,0.940062,49.0424,0.93037,0.938102,0.933027,"[11.372549019607849, 36.07843137254896, 56.078...",1,35.325
3,0.92282,142.954,0.914871,0.922365,0.91532,"[13.333333333333341, 38.039215686274446, 62.35...",1,39.6284
4,0.924218,183.717,0.90891,0.922562,0.908958,"[11.372549019607849, 36.07843137254896, 59.215...",1,41.1243
5,0.873706,624.08,0.865549,0.87101,0.863595,"[15.294117647058833, 39.215686274509736, 63.13...",0.666992,36.5415
6,0.899134,78.7307,0.893219,0.893835,0.893076,"[13.333333333333341, 38.039215686274446, 60.39...",0.75,41.4802
7,0.89477,24.4609,0.895363,0.889166,0.888735,"[23.13725490196077, 47.05882352941167, 71.3725...",1,50.7913
8,0.867637,211.883,0.856573,0.864156,0.854237,"[11.372549019607849, 35.29411764705877, 58.039...",0.777832,38.2456
9,0.917732,63.5224,0.907165,0.915032,0.907276,"[11.372549019607849, 36.07843137254896, 57.254...",1,46.4207


In [74]:
resultDataFrameWithPenalty_5Class_merged.to_pickle("1step-merged-5class-25_10Cons-Penalty")

In [77]:
resultDataFrameNoPenalty_5Class_merged.to_pickle("1step-merged-5class-25_10Cons")

In [79]:
name = "5normal_correlated_lumping_Boundaries.pickle"
with open(name, 'rb') as handle:
    normalLumping = pickle.load(handle)

In [81]:
percentageArray = []
for i in range(12):
    name = "./pickles/5normal_correlated_lumping_percentage_" + str(i) + ".pickle"
    with open(name, 'rb') as handle:
        percentageArray.append(pickle.load(handle))

In [82]:
accuracyResult = [np.inf for i in range(12)]
penaltyResult = [np.inf for i in range(12)]
precisionResult = [np.inf for i in range(12)]
recallResult = [np.inf for i in range(12)]
f1ScoreResult = [np.inf for i in range(12)]
boundariesResult = [np.inf for i in range(12)]
lumpAproxResult = [np.inf for i in range(12)]
lumpErrorResult = [np.inf for i in range(12)]


testDataFrame = {"accuracy": accuracyResult, "penalty": penaltyResult, "precision": precisionResult, 
                 "recall": recallResult, "f1Score": f1ScoreResult, "boundaries": boundariesResult, 
                 "lumpAprox": lumpAproxResult, "lumpError": lumpErrorResult}


resultDataFrameNoPenalty_7Class = pd.DataFrame(testDataFrame)
resultDataFrameNoPenalty_7Class = resultDataFrameNoPenalty_7Class.astype('object')

In [84]:
warnings.filterwarnings('ignore')
for timeIndex in range(12):
    if len(normalLumping[timeIndex]) == 0:
        continue
    
    print(timeIndex)
    boundaries = np.array([])
    for i in range(1, len(normalLumping[0][0][3])):
        bound = normalLumping[timeIndex][0][3][i] + 1
        counter = 0
        for j in range(len(percentageArray[timeIndex])):
            if percentageArray[timeIndex][j][2] == False:
                counter += 1
            if counter == bound:
                boundaries = np.append(boundaries, percentageArray[timeIndex][j][1])
                break
    boundaries = np.append(boundaries,100)
    print(boundaries)
    
    XArraysForLearning, YArraysForLearning, XArraysForTesting, YArraysForTesting, boundaries = dataPreparation(
        data, timeIndex, 30, boundaries, 1)

    print("starting tensor")
    lossFunctionBoolean = 0
    accuracy_result, penaltyValue, precision, recall, f1Score = tensorFlowLossFunction(
        lossFunctionBoolean, XArraysForLearning, YArraysForLearning, XArraysForTesting, 
        YArraysForTesting, boundaries)
    
    resultDataFrameNoPenalty_7Class.loc[timeIndex]["accuracy"] = accuracy_result
    resultDataFrameNoPenalty_7Class.loc[timeIndex]["penalty"] = penaltyValue
    resultDataFrameNoPenalty_7Class.loc[timeIndex]["precision"] = precision
    resultDataFrameNoPenalty_7Class.loc[timeIndex]["recall"] = recall
    resultDataFrameNoPenalty_7Class.loc[timeIndex]["f1Score"] = f1Score
    resultDataFrameNoPenalty_7Class.loc[timeIndex]["boundaries"] = boundaries
    resultDataFrameNoPenalty_7Class.loc[timeIndex]["lumpAprox"] = normalLumping[timeIndex][0][0]
    resultDataFrameNoPenalty_7Class.loc[timeIndex]["lumpError"] = normalLumping[timeIndex][0][1]
    print(resultDataFrameNoPenalty_7Class.loc[timeIndex])

    

0
[ 24.31372549  39.21568627  52.15686275  76.07843137  88.23529412
 100.        ]
start training set generation
375309
93828
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 26), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(27, 6) dtype=float32_ref>
Initialized
epoch  0 0.932034980388813 0.9356801807562775 12.322140744670032
epoch  20 0.9347928035470668 0.9325041565417572 10.96468919039911
epoch  40 0.9351045574693042 0.9346463742166518 10.9291948901558
epoch  60 0.9349739938608458 0.9352325531824189 10.917189686270685
epoch  80 0.9354322987721692 0.9359253101419619 10.889484710188375
epoch  100 0.9354669380968622 0.9350300549942447 10.898995604920042
epoch  120 0.9353576909959073 0.9334633584857399 10.886060404622993
epoch  140 0.9351285385402456 0.9313211408108454 10.877961774578978
epoch  160 0.9353337099249659 0.9351472907873982 10.890030620713677
e

epoch  160 0.9294198895027624 0.9240803368072331 12.860000616147378
epoch  180 0.9297479281767956 0.9248395334391607 12.84498120987612
epoch  200 0.9290400552486188 0.9230450686727862 12.818606505473022
0.9261508730761268
accuracy                                               0.926151
penalty                                                 149.617
precision                                               0.90867
recall                                                 0.924425
f1Score                                                0.909454
boundaries    [11.372549019607849, 36.07843137254896, 55.294...
lumpAprox                                                     1
lumpError                                               38.0115
Name: 4, dtype: object
5
[ 15.29411765  39.21568627  63.1372549   74.11764706  84.31372549
 100.        ]
start training set generation
137960
34491
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples.

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no true samples.



start training set generation
79152
19788
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 26), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(27, 6) dtype=float32_ref>
Initialized
epoch  0 0.8433328276699029 0.8761875884374368 27.53499165514913
epoch  20 0.8685022249190939 0.8786638366686881 21.595498205774426
epoch  40 0.8688056229773463 0.8757833030119264 21.49480820666624
epoch  60 0.8691216626213593 0.8786133009904993 21.452099721408594
epoch  80 0.8682241100323624 0.8745199110572064 21.425917159392217
epoch  100 0.8692354368932039 0.8749747321609056 21.41063213618442
epoch  120 0.8688941140776699 0.8785627653123105 21.41161394080682
epoch  140 0.8691848705501618 0.8772488376794016 21.389077207031402
epoch  160 0.8694377022653722 0.8793713361633313 21.388062367933113
0.8802809783707297


/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



accuracy                                               0.880281
penalty                                                 254.433
precision                                              0.872968
recall                                                 0.879321
f1Score                                                0.872097
boundaries    [18.039215686274513, 38.039215686274446, 48.23...
lumpAprox                                              0.588867
lumpError                                                38.103
Name: 10, dtype: object
11
[ 13.33333333  36.07843137  53.33333333  64.31372549  88.23529412
 100.        ]
start training set generation
60204
15051
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 26), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(27, 6) dtype=float32_ref>
Initialized
epoch  0 0.9126496010638298 0.9231944721280978 16.832026608700488
e

In [92]:
resultDataFrameNoPenalty_7Class.to_pickle("1step-merged-6class-25_10Cons")

# 60 min

In [101]:
data = pd.read_pickle("60min_featureEngineer.pickle")
# data.to_pickle("30min_featureEngineer.pickle")

In [110]:
name = "./pickles/5normal_25-10_lumping_60min.pickle"
with open(name, 'rb') as handle:
    normalLumping = pickle.load(handle)

In [111]:
percentageArray = []
for i in range(24):
    name = "./pickles/5normal_25-10_lumping_60min_percentage_" + str(i) + ".pickle"
    with open(name, 'rb') as handle:
        percentageArray.append(pickle.load(handle))

In [103]:
def dataPreparation(data, timeIndexes, minuteSplit, boundaries, steps):
    from sklearn.utils import shuffle
    warnings.filterwarnings('always')
    reg = "l2"
    solvers = "lbfgs"
    clf = LogisticRegression(penalty = reg, max_iter = 100000, random_state = 0,
                             solver = solvers , multi_class = 'multinomial')
    accuracyValue = 0
    numOfElements = 0
    f1scoreValue = 0
    precisionValue = 0
    recallValue = 0
    prevRowTrain = np.inf
    prevCU = np.inf

    sampleIntervals = 6 #seconds
    minuteSplit = 30 #minutes
    numOfSamples = minuteSplit * 60 / sampleIntervals
    # numberOfDays = len(numOfDays)
    days = np.zeros(7)
    numOfThirtyMinsPerDay = np.zeros(int((24 * 60) / minuteSplit)) #in this case 48
#     which6SecondsPerPeriod = np.zeros(int(minuteSplit * 60 / sampleIntervals)) #in this case 300
    prevRowTrain = np.inf
    prevCU = np.inf

    XArraysForLearning = []
    YArraysForLearning = []
    XArraysForTesting = []
    YArraysForTesting = []

    x = timeIndexes
    wholeDataFrame = data.loc[(data["timeIndex"] == x)].copy()
    
    wholeDataFrame["cuClass"] = wholeDataFrame["CU"].apply(lambda x: classifying(x, boundaries))

    

    stackCounter = 0
    prevCU = 0
    prev2CU = 0
    prev3CU = 0
    prev4CU = 0
    prev5CU = 0
    prevCUVal = 1
    print("start training set generation")
    #number of features are: prevCU + 48 correlations + 1 logDiff
    numberOfFeatures = 48 + 2*len(boundaries) + 2 + 24
    
    XArraysForLearning = np.zeros(shape=(len(wholeDataFrame), numberOfFeatures))
    YArraysForLearning = np.zeros(shape=(len(wholeDataFrame), len(boundaries)))
#     print(len(trainingDataFrame))
    counter = 0
    pandasIndexCounter = 0
    wholeDataFrame.reset_index()
    for index, row in wholeDataFrame.iterrows():
        if (index + steps) not in wholeDataFrame.index:
            continue
        booleanCont = 0
        for stationary in range(1, 48):
            if (index - stationary*steps) not in wholeDataFrame.index:
                booleanCont = 1
                continue
        if booleanCont == 1:
            continue
            
        XArraysForLearning[pandasIndexCounter, 0] = row["normalLogDiff"]
        for stationary in range(1, 48):
                XArraysForLearning[pandasIndexCounter, stationary] = wholeDataFrame.loc[index - stationary*steps]["normalLogDiff"]
                

        XArraysForLearning[pandasIndexCounter, 48 + wholeDataFrame.loc[index - steps]["cuClass"]] = 1
        XArraysForLearning[pandasIndexCounter, 48 + len(boundaries) + row["cuClass"]] = 1
        XArraysForLearning[pandasIndexCounter, 48 + 2 * len(boundaries)] = (wholeDataFrame.loc[index - steps]["CU"]/255)
        XArraysForLearning[pandasIndexCounter, 48 + 2 * len(boundaries) + 1] = (row["CU"] / 255)
        for corrs in range(24):
            XArraysForLearning[pandasIndexCounter, (48 + 2 * len(boundaries)) + 2 + corrs] = row["corr" + str(corrs + 1)]

#         diffLogValue = (np.log(row["CU"]) - np.log(wholeDataFrame.loc[index - steps]["CU"]))/ (np.log(255) - np.log(1))
        YArraysForLearning[pandasIndexCounter, wholeDataFrame.loc[index + steps]["cuClass"]] = 1
#         print(XArraysForLearning[pandasIndexCounter])

        which6SecondsPerPeriod = 0
    
        prevCU = row["cuClass"]
        prevCUVal = row["CU"]
        pandasIndexCounter += 1

    XArraysForLearning, YArraysForLearning = shuffle(XArraysForLearning, YArraysForLearning, random_state=0)
    XArraysForTraining = XArraysForLearning[:int(0.8*len(XArraysForLearning))]
    YArraysForTraining = YArraysForLearning[:int(0.8*len(YArraysForLearning))]
    XArraysForTesting = XArraysForLearning[int(0.8*len(XArraysForLearning)):]
    YArraysForTesting = YArraysForLearning[int(0.8*len(YArraysForLearning)):]
    print(XArraysForTraining.shape[0])
    print(XArraysForTesting.shape[0])


    return XArraysForTraining, YArraysForTraining, XArraysForTesting, YArraysForTesting, boundaries


def tensorFlowLossFunction(
    lossFuncBool, XArraysForLearning, YArraysForLearning, XArraysForTesting, YArraysForTesting, boundaries):
    
    CUDA_VISIBLE_DEVICES=""
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    os.environ['CUDA_VISIBLE_DEVICES'] = ''

    if tf.test.gpu_device_name():
        print('GPU found')
    else:
        print("No GPU found")
    
    batch_size = 64
    learning_rate = 0.01
    beta = 0.1
    numOfEpochs = 400
    graph = tf.Graph()
    with graph.as_default():
#         x = tf.placeholder(tf.float32, shape = (batch_size, XArraysForLearning.shape[1]))
#         y_ = tf.placeholder(tf.float32, shape = (batch_size, YArraysForLearning.shape[1]))
#         W = tf.Variable(tf.truncated_normal([XArraysForLearning.shape[1], YArraysForLearning.shape[1]], seed = 0), name="weights", dtype=tf.float32)
#         b = tf.Variable(tf.truncated_normal([YArraysForLearning.shape[1]], seed = 0), dtype=tf.float32)

        tf_test_dataset64 = tf.constant(XArraysForTesting)
        tf_test_dataset = tf.cast(tf_test_dataset64, tf.float32)

        
        x0 = tf.placeholder(tf.float32, shape = (batch_size, 48))
        x1 = tf.placeholder(tf.float32, shape = (batch_size, XArraysForLearning.shape[1] - 48))
        y0 = tf.placeholder(tf.float32, shape = (batch_size, XArraysForLearning.shape[1] - 47))
        y_ = tf.placeholder(tf.float32, shape = (batch_size, YArraysForLearning.shape[1]))
        W0 = tf.Variable(tf.truncated_normal([48, 1], seed = 0), name="weights0", dtype=tf.float32)
        W1 = tf.Variable(tf.truncated_normal([XArraysForLearning.shape[1] - 47, YArraysForLearning.shape[1]], seed = 0), name="weights1", dtype=tf.float32)
        b = tf.Variable(tf.truncated_normal([YArraysForLearning.shape[1]], seed = 0), dtype=tf.float32)
        print(x0)
        print(x1)
        print(W0)
        print(W1)
        
        result = tf.matmul(x0, W0)
#         print(y0)
        concated = tf.concat([result, x1], axis = 1)
#         print(concated)
        logits = tf.matmul(concated, W1)
        train_prediction = tf.nn.softmax(logits)
#         loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = y_)

        beta = 0.05
        
#         train_prediction = tf.nn.softmax(logits)
        # train_prediction = tf.nn.softmax_cross_entropy_with_logits_v2(labels = y_, logits = logits)
        testing = tf.concat([tf.matmul(tf_test_dataset[:,:48], W0), tf_test_dataset[:,48:]], axis = 1)
        test_prediction = tf.nn.softmax(tf.matmul(testing, W1))
        
#         logits = tf.matmul(x, W)

        # x = XArraysForLearning[0:(0 + batch_size), :]
        # y_ = tf.Variable(YArraysForLearning[0:(0 + batch_size), :])

        # loss = assymetricLossFunction(train_prediction, y_, boundaries)
        # loss = assymetricLossFunction(train_prediction, y_, boundaries)
        
        if lossFuncBool == 0:
            loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = y_)
        
        elif lossFuncBool == 1:
            loss = assymetricLossFunction(train_prediction, y_, boundaries)
#         dist = tfp.distributions.Multinomial(total_count=1, logits=logits)
#         loss = loss - (dist.log_prob(mult))
        # regularizer = tf.nn.l2_loss(W)
        # loss = tf.reduce_mean(loss + beta * regularizer)
        # loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits = train_prediction, labels = y_)
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

        prevAcc = 0
        prevLoss = np.inf

    with tf.Session(graph=graph) as session:
        initializer = tf.contrib.layers.xavier_initializer()
        tf.set_random_seed(0)
        W0 = tf.Variable(initializer([48, 1]))
        W1 = tf.Variable(initializer([XArraysForLearning.shape[1] - 47, YArraysForLearning.shape[1]]))
#         b = tf.Variable(initializer([YArraysForLearning.shape[1]]))       
        tf.global_variables_initializer().run()
#         print(W.eval())
        
        print("Initialized")

        numberOfBatchIteration = int(XArraysForLearning.shape[0] / batch_size)
        restOfData = XArraysForLearning.shape[0] % batch_size
        if restOfData != 0:
            numberOfBatchIteration += 1

        accuracy_result = 0
        accuracy_earlyStop = 0
        earlyStoppingCounter = 0
        
        for epoch in range(numOfEpochs):
            accuracyValue = 0
            lossValue = 0
            totalBatch = 0
            i = 0
            randomize = np.arange(XArraysForLearning.shape[0])
            random.Random(epoch).shuffle(randomize)
            XArraysForLearning = XArraysForLearning[randomize]
            YArraysForLearning = YArraysForLearning[randomize]

            for iteration in range(numberOfBatchIteration):
                if (iteration == numberOfBatchIteration - 1) and restOfData != 0:
                    break
                    batch_data = XArraysForLearning[i:, :]
                    batch_labels = YArraysForLearning[i:, :]

                else:
                    batch_data_x0 = XArraysForLearning[i:(i + batch_size), :48]
#                     print(batch_data_x0.shape)
                    batch_data_x1 = XArraysForLearning[i:(i + batch_size), 48:]
#                     print(batch_data_x1.shape)
                    batch_labels = YArraysForLearning[i:(i + batch_size), :]

                    i += batch_size


                feed_dict = {x0 : batch_data_x0, x1: batch_data_x1, y_ : batch_labels}
#                 print(y0.shape)
#                 print("aaaaaaaa")
#                 print(feed_dict)
                _, predictions, l = session.run([optimizer, train_prediction, loss], feed_dict=feed_dict)

#                 print(l)
                lossValue *= (iteration)
                lossValue += (np.sum(l))
                lossValue /= (iteration + 1)
                totalBatch += batch_size
#                     print(accuracy(batch_data, batch_labels))
                accuracyValue += accuracy(predictions, batch_labels) * batch_size
                
            testAcc = accuracy(test_prediction.eval(), YArraysForTesting)
            if testAcc < prevAcc and earlyStoppingCounter == 100:
                print(prevAcc)
                break

            elif testAcc < prevAcc:
                earlyStoppingCounter += 1

            elif testAcc >= prevAcc:
                prevAcc = testAcc
                earlyStoppingCounter = 0

            totalAccuracy = accuracyValue/totalBatch
#             print(session.run(W))
#             test_result = test_prediction.eval()
            if epoch % 20 == 0:
                print("epoch ", epoch, totalAccuracy, testAcc, lossValue)

#             print(accuracy(test_result, YArraysForTesting), prevAcc, earlyStoppingCounter)
                
            



        # print(session.run(W))
        predictionResult = test_prediction.eval()
        accResult = accuracy(predictionResult , YArraysForTesting)
        penaltyValue = assymetricPredictionScore(predictionResult, YArraysForTesting, boundaries)
#         print(W.eval())
        accuracy_result = prevAcc
        if accResult > prevAcc:
            accuracy_result = accResult
            
        correctLableIndex = np.argmax(YArraysForTesting, 1)
        predictionIndex = np.argmax(predictionResult, 1)
        
        precision = precision_score(correctLableIndex, predictionIndex, average='weighted')
        recall = recall_score(correctLableIndex, predictionIndex, average='weighted')
        f1Score = f1_score(correctLableIndex, predictionIndex, average='weighted')            
        
        return accuracy_result, penaltyValue, precision, recall, f1Score
            
    
def assymetricPredictionScore(predictedLables, trueLables, boundaries):
    xAxisPoints = np.linspace(rayleigh.ppf(0.01), rayleigh.ppf(0.99), 338)
    #number of overal datapoints must stay the same all the time
    maxState = 338

    inverseDistrib = max(rayleigh.pdf(xAxisPoints)) - rayleigh.pdf(xAxisPoints)
    minState = np.argmin(inverseDistrib)

    underUtilizedSum = 0
    overUtilizedSum = 0
    numberOfUnderUtilizedStates = minState - 0
    numberOfOverUtilizedStates = maxState - minState

    xAxisPoints -= xAxisPoints[np.argmin(inverseDistrib)]

    underUtilVal = numberOfUnderUtilizedStates / 100
    overUtilVal = numberOfOverUtilizedStates / 100

    correctLableIndex = np.argmax(trueLables, 1)
    predictionIndex = np.argmax(predictedLables, 1)

    diffPercentage = np.zeros(shape = (predictedLables.shape))

    penalties = np.zeros(shape = (predictedLables.shape))

    for index in range(predictedLables.shape[0]):
        diffPercentage[index] = boundaries[correctLableIndex[index]] - boundaries[:]


    for i in range(diffPercentage.shape[0]):
        for j in range(diffPercentage.shape[1]):
            if diffPercentage[i][j] > 0:
                penalties[i][j] = inverseDistrib[minState + math.floor
                                                  (diffPercentage[i][j] * overUtilVal)]
            else:
                penalties[i][j] = inverseDistrib[minState + math.floor
                                                  (diffPercentage[i][j] * underUtilVal)]

    sumOfPenalty = 0
    for i in range(predictionIndex.shape[0]):
        penalties[i] = penalties[i] / np.sum(penalties[i])
        sumOfPenalty += penalties[i][predictionIndex[i]]

    return sumOfPenalty
    
    
def accuracy(predictedLables, trueLables):
    import sys
    np.set_printoptions(threshold=sys.maxsize)
    correctLableIndex = np.argmax(trueLables, 1)
    predictionIndex = np.argmax(predictedLables, 1)
    errors = [0 for i in range(trueLables.shape[1])]
    corrects = [0 for i in range(trueLables.shape[1])]
    for i in range(len(correctLableIndex)):
        if correctLableIndex[i] != predictionIndex[i]:
            errors[correctLableIndex[i]] += 1
            corrects[predictionIndex[i]] += 1

    acc = np.float64(np.sum(correctLableIndex == predictionIndex)/predictedLables.shape[0])
    return acc
    
    
def assymetricLossFunction(prediction, correctLable, boundaries):
    xAxisPoints = np.linspace(rayleigh.ppf(0.01), rayleigh.ppf(0.99), 338)
    #number of overal datapoints must stay the same all the time
    maxState = 338
    inverseDistrib = max(rayleigh.pdf(xAxisPoints)) - rayleigh.pdf(xAxisPoints)
    inverseDistrib = tf.constant(inverseDistrib)
    xAxisPoints -= xAxisPoints[np.argmin(inverseDistrib)]
    minState = np.argmin(inverseDistrib)
    numberOfOverUtilizedStates = maxState - minState
    numberOfUnderUtilizedStates = minState
    minState = tf.constant(minState, tf.float32)
    numberOfOverUtilizedStates = tf.constant(numberOfOverUtilizedStates, tf.float32)
    numberOfUnderUtilizedStates = tf.constant(numberOfUnderUtilizedStates, tf.float32)

    underUtilVal = numberOfUnderUtilizedStates / 100
    overUtilVal = numberOfOverUtilizedStates / 100


    boundaries = tf.constant(boundaries, tf.float32)
    correctLableIndex = tf.argmax(correctLable, 1)

    diffPercentage = []
    for index in range(correctLableIndex.shape[0]):
        diffPercentage.append(boundaries[correctLableIndex[index]] - boundaries[:])

    diffPercentage = tf.stack(diffPercentage)

    penalties = []

    counter = 0
    for i in range(diffPercentage.shape[0]):
        for j in range(diffPercentage.shape[1]):
            counter += 1

            penalties.append(tf.cond(
                    tf.greater(diffPercentage[i][j], 0),
                    lambda: inverseDistrib[tf.dtypes.cast(minState + tf.math.floor
                                                                   (tf.math.scalar_mul(diffPercentage[i][j],
                                                                                       overUtilVal)), tf.int32)],
                    lambda: inverseDistrib[tf.dtypes.cast(minState + tf.math.floor
                                                                   (tf.math.scalar_mul
                                                                    (diffPercentage[i][j], underUtilVal))
                                                                   , tf.int32)]
                    ))

    penalties = tf.stack(penalties)
    penalties = tf.dtypes.cast(penalties, tf.float32)
    penalties = tf.reshape(penalties, diffPercentage.shape)
#     penalties = penalties / tf.norm(penalties)

    # weights = tf.reduce_sum(penalties * (1-prediction), axis=1)
    weights = (1 - penalties) * prediction
    weights = weights / tf.norm(weights)
    # print(correctLable)
    # print(prediction)
    # print(penalties)
    loss = tf.losses.softmax_cross_entropy(onehot_labels = correctLable, logits = weights)
    # weighted_losses = tf.nn.softmax_cross_entropy_with_logits_v2(labels = penalties, logits = prediction)
    # loss = tf.reduce_sum(weighted_losses)
    # loss = tf.reduce_sum(penalties * prediction)
    return tf.reduce_mean(loss)

In [105]:
accuracyResult = [np.inf for i in range(len(normalLumping))]
penaltyResult = [np.inf for i in range(len(normalLumping))]
precisionResult = [np.inf for i in range(len(normalLumping))]
recallResult = [np.inf for i in range(len(normalLumping))]
f1ScoreResult = [np.inf for i in range(len(normalLumping))]
boundariesResult = [np.inf for i in range(len(normalLumping))]
lumpAproxResult = [np.inf for i in range(len(normalLumping))]
lumpErrorResult = [np.inf for i in range(len(normalLumping))]


testDataFrame = {"accuracy": accuracyResult, "penalty": penaltyResult, "precision": precisionResult, 
                 "recall": recallResult, "f1Score": f1ScoreResult, "boundaries": boundariesResult, 
                 "lumpAprox": lumpAproxResult, "lumpError": lumpErrorResult}

resultDataFrameWithPenalty_5_60 = pd.DataFrame(testDataFrame)
resultDataFrameWithPenalty_5_60 = resultDataFrameWithPenalty_5_60.astype('object')

In [ ]:
warnings.filterwarnings('ignore')
for timeIndex in range(24):
    if len(normalLumping[timeIndex]) == 0:
        continue
#     print(normalLumping[timeIndex][0][3])
    print(timeIndex)
    boundaries = np.array([])
    for i in range(1, len(normalLumping[0][0][3])):
        bound = normalLumping[timeIndex][0][3][i] + 1
        counter = 0
        for j in range(len(percentageArray[timeIndex])):
            if percentageArray[timeIndex][j][2] == False:
                counter += 1
            if counter == bound:
                boundaries = np.append(boundaries, percentageArray[timeIndex][j][1])
                break
    boundaries = np.append(boundaries,100)
    print(boundaries)

    XArraysForLearning, YArraysForLearning, XArraysForTesting, YArraysForTesting, boundaries = dataPreparation(
        data, timeIndex, 30, boundaries, 1)

    print("starting tensor")
    lossFunctionBoolean = 1
    accuracy_result, penaltyValue, precision, recall, f1Score = tensorFlowLossFunction(
        lossFunctionBoolean, XArraysForLearning, YArraysForLearning, XArraysForTesting, 
        YArraysForTesting, boundaries)
    
# #     print(accuracy_result, penaltyValue)
    resultDataFrameWithPenalty_5_60.loc[timeIndex]["accuracy"] = accuracy_result
    resultDataFrameWithPenalty_5_60.loc[timeIndex]["penalty"] = penaltyValue
    resultDataFrameWithPenalty_5_60.loc[timeIndex]["precision"] = precision
    resultDataFrameWithPenalty_5_60.loc[timeIndex]["recall"] = recall
    resultDataFrameWithPenalty_5_60.loc[timeIndex]["f1Score"] = f1Score
    resultDataFrameWithPenalty_5_60.loc[timeIndex]["boundaries"] = boundaries
    resultDataFrameWithPenalty_5_60.loc[timeIndex]["lumpAprox"] = normalLumping[timeIndex][0][0]
    resultDataFrameWithPenalty_5_60.loc[timeIndex]["lumpError"] = normalLumping[timeIndex][0][1]
    print(resultDataFrameWithPenalty_5_60.loc[timeIndex])

0
[ 10.19607843  32.15686275  51.37254902  71.37254902  82.35294118
 100.        ]
start training set generation
37896
9475
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 38), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(39, 6) dtype=float32_ref>
Initialized
epoch  0 0.8867715371621622 0.9119788918205805 1.2621315658293866
epoch  20 0.9167282516891891 0.9198944591029024 1.2581605519595986
epoch  40 0.9154613597972973 0.916622691292876 1.2578094253266185
epoch  60 0.9164379222972973 0.9187335092348285 1.2578247801677613
epoch  80 0.9169394003378378 0.9167282321899736 1.2578409000022983
epoch  100 0.9170977618243243 0.9167282321899736 1.2577228875377702
epoch  120 0.9164379222972973 0.9198944591029024 1.2577243800702924
epoch  140 0.9163851351351351 0.915778364116095 1.2577817867937933
epoch  160 0.9164643158783784 0.9187335092348285 1.257865989530408
ep

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



accuracy                                               0.922005
penalty                                                 54.3654
precision                                              0.913811
recall                                                 0.918628
f1Score                                                0.912551
boundaries    [10.196078431372554, 32.156862745097996, 51.37...
lumpAprox                                                     1
lumpError                                               44.9435
Name: 0, dtype: object
1
[ 10.19607843  34.11764706  48.23529412  59.21568627  76.07843137
 100.        ]
start training set generation
38177
9545
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 38), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(39, 6) dtype=float32_ref>
Initialized
epoch  0 0.8925912332214765 0.9158721843897328 1.240799583644676
epoch

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.



accuracy                                               0.925511
penalty                                                 69.6957
precision                                              0.913377
recall                                                 0.923415
f1Score                                                0.917253
boundaries    [10.196078431372554, 34.11764705882348, 48.235...
lumpAprox                                                     1
lumpError                                               22.8649
Name: 1, dtype: object
2
[ 24.31372549  42.35294118  54.11764706  64.31372549  78.03921569
 100.        ]
start training set generation
37746
9437
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 38), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(39, 6) dtype=float32_ref>
Initialized
epoch  0 0.9269153225806451 0.956024160220409 1.2582098282051417
epoch

...

accuracy                                               0.929482
penalty                                                 54.3398
precision                                              0.922921
recall                                                 0.927232
f1Score                                                0.923309
boundaries    [10.196078431372554, 34.11764705882348, 51.372...
lumpAprox                                                     1
lumpError                                               21.7719
Name: 5, dtype: object
6
[ 10.19607843  32.15686275  50.19607843  60.39215686  76.07843137
 100.        ]
start training set generation
36891
9223
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 38), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(39, 6) dtype=float32_ref>
Initialized
epoch  0 0.8933919270833334 0.9183562832050309 1.253994976066882
epoch

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples.

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no true samples.



accuracy                                               0.926922
penalty                                                 59.8845
precision                                              0.920162
recall                                                 0.925729
f1Score                                                0.920469
boundaries    [10.196078431372554, 32.156862745097996, 50.19...
lumpAprox                                                     1
lumpError                                                 27.74
Name: 6, dtype: object
7
[ 11.37254902  36.07843137  50.19607843  63.1372549   81.17647059
 100.        ]
start training set generation
36717
9180
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 38), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(39, 6) dtype=float32_ref>
Initialized
epoch  0 0.8854439354275742 0.9119825708061002 1.3073054470941037
epoc

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples.

/home/sepehr/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no true samples.



accuracy                                               0.913756
penalty                                                 86.2771
precision                                              0.906004
recall                                                   0.9116
f1Score                                                0.906477
boundaries    [13.333333333333341, 37.25490196078425, 60.392...
lumpAprox                                                     1
lumpError                                                40.969
Name: 8, dtype: object
9
[ 11.37254902  36.07843137  49.01960784  67.05882353  81.17647059
 100.        ]
start training set generation
37436
9360
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 38), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(39, 6) dtype=float32_ref>
Initialized
epoch  0 0.8814212328767124 0.9076923076923077 1.3436023952209781
epoc

In [371]:
resultDataFrameWithPenalty_5_60

NameError: name 'resultDataFrameWithPenalty_5_60' is not defined

In [114]:
resultDataFrameWithPenalty_5_60.to_pickle("1step-60min-6class-25_10Cons-Penalty")

In [108]:
from multiprocessing import Pool
p = multiprocessing.Pool(multiprocessing.cpu_count())
listOfTimeIndexes = [i for i in range(24)]
p.map(testing, listOfTimeIndexes)

0
8
2
6
1
7
5
4
3
16
15
21
10
22
20
18
[ 11.37254902  36.07843137  55.29411765  79.21568627 100.        ]
23
11
14
12
17
9
19
13
[ 24.31372549  35.29411765  51.37254902  75.29411765 100.        ]
[ 10.19607843  32.15686275  51.37254902  76.07843137 100.        ]
[ 11.37254902  35.29411765  59.21568627  81.17647059 100.        ]
[ 11.37254902  36.07843137  60.39215686  83.1372549  100.        ]
[ 20.39215686  45.09803922  64.31372549  88.23529412 100.        ]
[ 24.31372549  42.35294118  54.11764706  78.03921569 100.        ]
[ 22.35294118  44.31372549  67.05882353  86.2745098  100.        ]
[ 11.37254902  36.07843137  55.29411765  76.07843137 100.        ]
[ 24.31372549  49.01960784  61.17647059  84.31372549 100.        ]
[ 11.37254902  36.07843137  60.39215686  78.03921569 100.        ]
[ 13.33333333  37.25490196  60.39215686  78.03921569 100.        ]
[ 10.19607843  32.15686275  56.07843137  78.03921569 100.        ]
[ 10.19607843  34.11764706  51.37254902  76.07843137 100.        ]


<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(37, 5) dtype=float32_ref>
38352
9589
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 36), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(37, 5) dtype=float32_ref>
38412
9604
starting tensor
No GPU found
Tensor("Placeholder:0", shape=(64, 48), dtype=float32)
Tensor("Placeholder_1:0", shape=(64, 36), dtype=float32)
<tf.Variable 'weights0:0' shape=(48, 1) dtype=float32_ref>
<tf.Variable 'weights1:0' shape=(37, 5) dtype=float32_ref>


Process ForkPoolWorker-546:
Process ForkPoolWorker-560:
Process ForkPoolWorker-549:
Process ForkPoolWorker-555:


KeyboardInterrupt: 

Process ForkPoolWorker-556:
Process ForkPoolWorker-554:
Process ForkPoolWorker-550:
Process ForkPoolWorker-553:
Process ForkPoolWorker-545:
Process ForkPoolWorker-547:
Process ForkPoolWorker-552:
Process ForkPoolWorker-558:
Process ForkPoolWorker-548:
Process ForkPoolWorker-557:
Process ForkPoolWorker-559:
Process ForkPoolWorker-551:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  Fi

  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multipro

In [ ]:
warnings.filterwarnings('ignore')
for timeIndex in range(4):
    if len(normalLumping[timeIndex]) == 0:
        continue
    
    print(timeIndex)
    boundaries = np.array([])
    for i in range(1, len(normalLumping[0][0][3])):
        bound = normalLumping[timeIndex][0][3][i] + 1
        counter = 0
        for j in range(len(percentageArray[timeIndex])):
            if percentageArray[timeIndex][j][2] == False:
                counter += 1
            if counter == bound:
                boundaries = np.append(boundaries, percentageArray[timeIndex][j][1])
                break
    boundaries = np.append(boundaries,100)
    print(boundaries)
    
#     XArraysForLearning, YArraysForLearning, XArraysForTesting, YArraysForTesting, boundaries = dataPreparation(
#         data, timeIndex, 30, boundaries)

    print("starting tensor")
    lossFunctionBoolean = 1
    accuracy_result, penaltyValue, precision, recall, f1Score = tensorFlowLossFunction(
        lossFunctionBoolean, XArraysForLearning, YArraysForLearning, XArraysForTesting, 
        YArraysForTesting, boundaries)
    
    resultDataFrameWithPenalty_5Class.loc[timeIndex]["accuracy"] = accuracy_result
    resultDataFrameWithPenalty_5Class.loc[timeIndex]["penalty"] = penaltyValue
    resultDataFrameWithPenalty_5Class.loc[timeIndex]["precision"] = precision
    resultDataFrameWithPenalty_5Class.loc[timeIndex]["recall"] = recall
    resultDataFrameWithPenalty_5Class.loc[timeIndex]["f1Score"] = f1Score
    resultDataFrameWithPenalty_5Class.loc[timeIndex]["boundaries"] = boundaries
    resultDataFrameWithPenalty_5Class.loc[timeIndex]["lumpAprox"] = normalLumping[timeIndex][0][0]
    resultDataFrameWithPenalty_5Class.loc[timeIndex]["lumpError"] = normalLumping[timeIndex][0][1]
    print(resultDataFrameWithPenalty_5Class.loc[timeIndex])

    

0
[ 24.31372549  39.21568627  56.07843137  77.25490196 100.        ]
starting tensor
Initialized
epoch  0 0.9443505925989086 1.5203611499656582
epoch  20 0.9507935069918144 1.5180442087240507
epoch  40 0.9507402157230559 1.5180384899917783
epoch  60 0.9508254817530696 1.51802356600111
epoch  80 0.951062627899045 1.5180116068739509
epoch  100 0.9508947604024557 1.51801653362882
epoch  120 0.9511212482946794 1.5180158692159256
epoch  140 0.9508201526261937 1.5180075952496086
epoch  160 0.9510839444065484 1.518003221321722
epoch  180 0.9512571410300137 1.5180000742899944
epoch  200 0.951099931787176 1.5179998530494252
epoch  220 0.9512651347203275 1.5179923774234858
epoch  240 0.9511532230559345 1.5179967585478402
epoch  260 0.9511079254774898 1.5179891945110644
epoch  280 0.9511745395634379 1.5179859068839334
epoch  300 0.9512518119031378 1.5179849997223802
epoch  320 0.9511558876193724 1.5179818886933167
epoch  340 0.9514116857094134 1.5179778790815484
epoch  360 0.9513850400750341 1.51

In [23]:
resultDataFrameWithPenalty_5Class

,accuracy,penalty,precision,recall,f1Score,boundaries,lumpAprox,lumpError
0,0.951347,464.175,0.951195,0.951347,0.951162,"[24.31372549019606, 39.215686274509736, 56.078...",1,42.0209
1,0.921882,58.8915,0.917558,0.921882,0.918629,"[10.196078431372554, 33.333333333333286, 54.11...",1,46.7483
2,0.934836,37.4196,0.93411,0.934836,0.933227,"[11.372549019607849, 36.07843137254896, 56.078...",1,35.325
3,0.918646,127.284,0.913256,0.918646,0.912231,"[13.333333333333341, 38.039215686274446, 62.35...",1,39.6284
4,0.891849,142.988,0.892459,0.891849,0.885425,"[11.372549019607849, 36.07843137254896, 59.215...",1,41.1243
5,0.870459,564.53,0.858941,0.870459,0.862781,"[15.294117647058833, 39.215686274509736, 63.13...",0.666992,36.5415
6,0.875994,85.6738,0.873454,0.875994,0.873438,"[13.333333333333341, 38.039215686274446, 60.39...",0.75,41.4802
7,0.845579,31.848,0.849369,0.845579,0.844703,"[23.13725490196077, 47.05882352941167, 71.3725...",1,50.7913
8,0.88397,150.218,0.878757,0.88397,0.87878,"[11.372549019607849, 35.29411764705877, 58.039...",0.777832,38.2456
9,0.90297,67.6481,0.89888,0.90297,0.899907,"[11.372549019607849, 36.07843137254896, 57.254...",1,46.4207
